**Lisa Ho Yen Xin** 
**23104045**

**Code for Web Scrap**

# Mudah.my

## Scrap General Info

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import pandas as pd

def get_url(base_url, page):
    return f"{base_url}?o={page}"  # Mudah uses ?o= for pagination

def scrape(main_url, pages, output_csv):
    driver = webdriver.Chrome()

    data = []

    for page in range(1, pages + 1):
        print(f"Scraping page {page}...")
        url = get_url(main_url, page)
        driver.get(url)
        time.sleep(3)

        listings = driver.find_elements(By.XPATH, "//div[contains(@class, 'w_100%')]/a")

        for listing in listings:
            try:
                price = listing.find_element(By.XPATH, './/span[contains(@class, "currPrice")]').text
                location = listing.find_element(By.XPATH, './/h3[contains(@class, "c_black")]').text
                posted = listing.find_element(By.XPATH, './/p').text

                details = listing.find_elements(By.XPATH, './/span[contains(@class, "fw_bold")]')
                size = details[0].text if len(details) > 0 else ''
                bedrooms = details[1].text if len(details) > 1 else ''
                bathrooms = details[2].text if len(details) > 2 else ''

                tenure = ''
                all_spans = listing.find_elements(By.XPATH, './/span')
                for span in all_spans:
                    text = span.text.strip().lower()
                    if "freehold" in text or "leasehold" in text:
                        tenure = span.text.strip()
                        break

                listing_url = listing.get_attribute('href')

                data.append({
                    "Price": price,
                    "Location": location,
                    "Posted": posted,
                    "Size (sqft)": size,
                    "Bedrooms": bedrooms,
                    "Bathrooms": bathrooms,
                    "Tenure": tenure,
                    "URL": listing_url
                })

            except (StaleElementReferenceException, NoSuchElementException):
                continue

    driver.quit()

    # Clean: Remove rows with all empty values
    df = pd.DataFrame(data)
    df = df.dropna(how='all')

    df.to_csv(output_csv, index=False)
    print(f"✅ Data saved to '{output_csv}'.")


### Selangor - Apartment & Condominium

In [3]:
 if __name__ == "__main__":
    main_url = "https://www.mudah.my/selangor/apartment-condominium-for-sale"
    total_pages = 25
    output_csv="mudah_selangor_apartment_condominium_data.csv"
    scrape(main_url, total_pages, output_csv)


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
✅ Data saved to 'mudah_selangor_apartment_condominium_data.csv'.


### Selangor - House

In [12]:
 if __name__ == "__main__":
    main_url = "https://www.mudah.my/selangor/houses-for-sale"
    total_pages = 15
    output_csv="mudah_selangor_house_data.csv"
    scrape(main_url, total_pages, output_csv)


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
✅ Data saved to 'mudah_selangor_house_data.csv'.


### KL - Apartment & Condominium

In [8]:
if __name__ == "__main__":
    main_url = "https://www.mudah.my/kuala-lumpur/apartment-condominium-for-sale"
    total_pages = 15
    output_csv = "mudah_kl_apartment_condominium_data.csv"
    scrape(main_url, total_pages, output_csv)


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
✅ Data saved to 'mudah_kl_apartment_condominium_data.csv'.


### KL - House

In [13]:
if __name__ == "__main__":
    main_url = "https://www.mudah.my/kuala-lumpur/houses-for-sale"
    total_pages = 15
    output_csv = "mudah_kl_house_data.csv"
    scrape(main_url, total_pages, output_csv)


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
✅ Data saved to 'mudah_kl_house_data.csv'.


## Scrap Facilities

In [1]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def init_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--log-level=3")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

def scrape_facilities(driver, url, retries=3):
    for attempt in range(retries):
        try:
            driver.get(url)
            wait = WebDriverWait(driver, 10)

            # Locate the "Facilities" heading <p>
            heading_elem = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//p[text()='Facilities']")
            ))

            # Find the parent or the next container after the heading
            # Assuming it's the parent div that contains the heading and the facilities list
            parent_div = heading_elem.find_element(By.XPATH, "./ancestor::div[1]")

            # Then find all <p> tags under this container excluding the heading itself
            facilities = [elem.text.strip() for elem in parent_div.find_elements(By.TAG_NAME, "p") if elem.text.strip() and elem.text.strip() != "Facilities"]

            return ", ".join(facilities) if facilities else "Not Listed"

        except TimeoutException:
            print(f"[Timeout] Attempt {attempt+1} for {url}")
            if attempt == retries - 1:
                return "Timeout"
            time.sleep(3)

        except NoSuchElementException:
            print(f"[NoSuchElement] Facilities block not found in {url}")
            return "Not Found"

        except Exception as e:
            print(f"[Error] {e} in {url}")
            return "Error"
    return "Failed"


def add_facilities_to_dataset(input_csv, output_csv, headless=True):
    df = pd.read_csv(input_csv)
    driver = init_driver(headless=headless)
    facilities_list = []

    for idx, url in enumerate(df['URL']):
        print(f"Scraping ({idx + 1}/{len(df)}): {url}")
        facilities = scrape_facilities(driver, url)
        facilities_list.append(facilities)

        # Random short delay to avoid detection
        time.sleep(random.uniform(0.5, 1.5))

        # Save progress periodically
        if (idx + 1) % 10 == 0:
            df['Facilities'] = facilities_list + [""] * (len(df) - len(facilities_list))
            df.to_csv(output_csv, index=False)
            print(f"Progress saved at {idx + 1} listings")

    df['Facilities'] = facilities_list
    df.to_csv(output_csv, index=False)
    driver.quit()
    print(f"Scraping completed. Output saved to {output_csv}")

### Selangor - Apartment & Condominium

In [4]:
# scrap selangor facilities
input_csv = "mudah_selangor_apartment_condominium_data.csv"
output_csv = "mudah_selangor_apartment_condominium_with_facilities.csv"
add_facilities_to_dataset(input_csv, output_csv, headless=True)


Scraping (1/1075): https://www.mudah.my/pangsapuri-taman-kajang-mewah-cheapest-price-than-lelong-low-booking-110850394.htm
Scraping (2/1075): https://www.mudah.my/pj-ara-damansara-next-lrt-mall-above-shop-lot-full-furnished-111201365.htm
Scraping (3/1075): https://www.mudah.my/residensi-zamrud-kajang-renovate-111014753.htm
Scraping (4/1075): https://www.mudah.my/100-loan-d-cassia-apartment-850sf-setia-ecohil-semenyih-freehold-111137535.htm
Scraping (5/1075): https://www.mudah.my/100-loan-vesta-view-apartment-1044sf-taman-putra-impian-kajang-111137536.htm
Scraping (6/1075): https://www.mudah.my/full-loan-freehold-renovate-casa-puteri-condo-bandar-puteri-puchong-111280441.htm
Scraping (7/1075): https://www.mudah.my/100-loan-pangsapuri-aranda-750sf-shah-alam-below-market-price-111280449.htm
Scraping (8/1075): https://www.mudah.my/full-loan-renovated-koi-tropika-bandar-puteri-puchong-110880359.htm
Scraping (9/1075): https://www.mudah.my/mentari-court-bandar-sunway-109821573.htm
[Timeout] A

Scraping (58/1075): https://www.mudah.my/cashback-depo1k-full-loan-sd-apartment-2-kepong-bandar-sri-damansara-108173716.htm
Scraping (59/1075): https://www.mudah.my/for-sale-with-tenancy-111234588.htm
Scraping (60/1075): https://www.mudah.my/f-loan-gatedguard-mv-simfoni-apartment-eco-majestic-kajang-semenyih-111165236.htm
Progress saved at 60 listings
Scraping (61/1075): https://www.mudah.my/t-parkland-rawang-condo-3r3b-side-by-side-parking-111234773.htm
Scraping (62/1075): https://www.mudah.my/impian-heights-fully-furnished-unit-for-sale-111230685.htm
Scraping (63/1075): https://www.mudah.my/apartment-pusat-hentian-kajang-111233900.htm
Scraping (64/1075): https://www.mudah.my/shang-villa-condo-kelana-jaya-freehold-block-a-middle-floor-111280156.htm
Scraping (65/1075): https://www.mudah.my/condo-for-sale-at-tropicana-aman-1-urban-homes-block-a-3a-08-111234786.htm
Scraping (66/1075): https://www.mudah.my/cashback-full-loan-lower-floor-rista-villa-taman-putra-perdana-puchong-110299274.ht

Scraping (124/1075): https://www.mudah.my/actual-price-vesta-view-taman-putra-impian-kajang-955sf-100-fulloan-108435283.htm
Scraping (125/1075): https://www.mudah.my/booking-1k-vista-millennium-nego-diskaun-80k-puchong-perdana-110188288.htm
Scraping (126/1075): https://www.mudah.my/2-carpark-reno-balcony-amara-residence-condo-batu-caves-sri-gombak-102330063.htm
Scraping (127/1075): https://www.mudah.my/combo-dapat-rumah-penyewa-melayu-flat-di-pusat-hentian-kajang-104374666.htm
Scraping (128/1075): https://www.mudah.my/nego-fully-furnish-full-reno-d-tinggian-suasana-cheras-selangor-106898218.htm
Scraping (129/1075): https://www.mudah.my/lily-rose-apartment-kelana-jaya-save-100k-freehold-renovated-nice-109890125.htm
[Timeout] Attempt 1 for https://www.mudah.my/lily-rose-apartment-kelana-jaya-save-100k-freehold-renovated-nice-109890125.htm
[Timeout] Attempt 2 for https://www.mudah.my/lily-rose-apartment-kelana-jaya-save-100k-freehold-renovated-nice-109890125.htm
[Timeout] Attempt 3 for ht

Scraping (186/1075): https://www.mudah.my/murah-apartment-cheras-utama-pangsapuri-alam-damai-kuala-lumpur-111278927.htm
Scraping (187/1075): https://www.mudah.my/booking-1k-centrestage-designer-suite-nego-diskaun-80k-full-loan-111278675.htm
Scraping (188/1075): https://www.mudah.my/murah-apartment-damai-bandar-sri-subang-petaling-jaya-111278668.htm
Scraping (189/1075): https://www.mudah.my/flat-apartment-taman-bukit-mutiara-kajang-freehold-111278650.htm
[Timeout] Attempt 1 for https://www.mudah.my/flat-apartment-taman-bukit-mutiara-kajang-freehold-111278650.htm
[Timeout] Attempt 2 for https://www.mudah.my/flat-apartment-taman-bukit-mutiara-kajang-freehold-111278650.htm
[Timeout] Attempt 3 for https://www.mudah.my/flat-apartment-taman-bukit-mutiara-kajang-freehold-111278650.htm
Scraping (190/1075): https://www.mudah.my/duplex-upper-lower-floor-111278649.htm
Progress saved at 190 listings
Scraping (191/1075): https://www.mudah.my/100-full-loan-low-depo-pangsapuri-prima-cempaka-kajang-pri

Scraping (250/1075): https://www.mudah.my/jalan-kaki-ke-lrt-cahaya-apartment-taman-cahaya-ada-lif-109503387.htm
Progress saved at 250 listings
Scraping (251/1075): https://www.mudah.my/termurah-1-130-sq-ft-suri-puteri-apartment-seksyen-20-shah-alam-111017023.htm
Scraping (252/1075): https://www.mudah.my/100-loan-guaranteed-de-bayu-setia-alam-2parkings-0-deposit-111278131.htm
Scraping (253/1075): https://www.mudah.my/skim-100-loan-ascotte-boulevard-free-hold-unit-860sqft-i-3r2b-2cp-109541713.htm
Scraping (254/1075): https://www.mudah.my/termurahhh-sri-permata-condo-seksyen-9-shah-alam-111278263.htm
Scraping (255/1075): https://www.mudah.my/well-kept-residensi-adelia-bangi-avenue-1082sqft-3r3b-107367026.htm
Scraping (256/1075): https://www.mudah.my/skim-100-loan-residensi-aman-semenyih-3r2b-freehold-unit-107366993.htm
Scraping (257/1075): https://www.mudah.my/corner-unit-non-bumi-lot-park-avenue-condominium-damansara-damai-111278246.htm
Scraping (258/1075): https://www.mudah.my/level-1-a

...

Scraping (824/1075): https://www.mudah.my/cheap-below-market-apartment-kelisa-ria-kajang-near-to-ukm-ktm-111271960.htm
Scraping (825/1075): https://www.mudah.my/pool-view-radia-residence-bukit-jelutong-108528087.htm
Scraping (826/1075): https://www.mudah.my/corner-unit-palmyra-residence-tingkat-19-bangi-105786903.htm
Scraping (827/1075): https://www.mudah.my/palmyra-residence-corner-lot-block-b-bandar-puteri-bangi-selangor-111028230.htm
Scraping (828/1075): https://www.mudah.my/ground-floor-laman-suria-apartment-kajang-utama-108555795.htm
Scraping (829/1075): https://www.mudah.my/full-loan-100-pangsapuri-saujana-permai-2-mutiara-height-kajang-111271871.htm
Scraping (830/1075): https://www.mudah.my/new-renovated-near-mrt-corner-pelangi-damansara-condominium-for-sale-111271800.htm
Progress saved at 830 listings
Scraping (831/1075): https://www.mudah.my/termurah-level-3-jalan-kaki-ke-mrt-lestari-apartment-damansara-damai-111271862.htm
Scraping (832/1075): https://www.mudah.my/landmark-res

Scraping (883/1075): https://www.mudah.my/suriamas-condominium-bandar-sunway-pj-2-parking-111271037.htm
Scraping (884/1075): https://www.mudah.my/renovated-vista-lavender-apartment-seksyen-3-taman-kinrara-puchong-111271162.htm
Scraping (885/1075): https://www.mudah.my/pangsapuri-suria-perdana-fortune-park-5-min-to-the-mines-111270983.htm
Scraping (886/1075): https://www.mudah.my/best-rent-at-below-value-cyberia-smarthome-condo-0-down-cyberjaya-110820005.htm
Scraping (887/1075): https://www.mudah.my/taman-cheras-mas-flat-111271060.htm
Scraping (888/1075): https://www.mudah.my/raya-apartment-bandar-country-homes-rawang-74299080.htm
Scraping (889/1075): https://www.mudah.my/taman-orkid-cheras-sering-akasia-tiles-flooring-freehold-for-sale-111271054.htm
Scraping (890/1075): https://www.mudah.my/pangsapuri-saujana-permai-1-apartment-prima-saujana-untung-beli-110873733.htm
Progress saved at 890 listings
Scraping (891/1075): https://www.mudah.my/100-loan-lppsa-kemuning-aman-apartment-shah-ala

Scraping (935/1075): https://www.mudah.my/below-market-the-boulevard-persiaran-wangsa-baiduri-subang-jaya-108336233.htm
Scraping (936/1075): https://www.mudah.my/nice-lift-desa-mas-apartment-t1-bandar-countrt-homes-rawang-110538093.htm
Scraping (937/1075): https://www.mudah.my/1st-floor-renovated-unit-cyber-heights-villa-apartment-cyberjaya-111196670.htm
Scraping (938/1075): https://www.mudah.my/tingkat-3-rumah-flat-sri-mutiara-100-loan-taman-ukay-bistari-110266543.htm
Scraping (939/1075): https://www.mudah.my/sri-dahlia-apartment-100-loan-tingkat-3-bandar-puteri-puchong-105105910.htm
Scraping (940/1075): https://www.mudah.my/100-loan-lppsa-pangsapuri-de-bayu-setia-alam-2park-murah-cashback50k-108345683.htm
Progress saved at 940 listings
Scraping (941/1075): https://www.mudah.my/nilam-puri-condo-bukit-puchong-915sqft-100-full-loan-0-deposit-108916497.htm
Scraping (942/1075): https://www.mudah.my/banjaria-court-gombak-991sqft-gombak-100-full-loan-0-deposit-106046227.htm
Scraping (943/10

Progress saved at 1000 listings
Scraping (1001/1075): https://www.mudah.my/tingkat-2-teratai-apartment-100-loan-taman-putra-perdana-puchong-104428517.htm
Scraping (1002/1075): https://www.mudah.my/desa-saujana-apartment-fully-reno-tingkat-1-taman-sungai-besi-indah-108043717.htm
Scraping (1003/1075): https://www.mudah.my/pool-view-renovated-1036sf-unit-kelana-puteri-condo-kelana-jaya-107156502.htm
Scraping (1004/1075): https://www.mudah.my/5min-midvalley-walking-2-school-restaurant-mrt-fully-furnished-111269543.htm
Scraping (1005/1075): https://www.mudah.my/100-fullloan-lppsa-mutiara-ville-cyberjaya-1180sf-0-depo-cashback50k-110713827.htm
Scraping (1006/1075): https://www.mudah.my/sun-u-residence-bandar-sunway-4-rooms-1-453sf-9-roi-pj-condo-109765479.htm
Scraping (1007/1075): https://www.mudah.my/full-loan-level-1-amazing-heights-apartment-sg-udang-klang-110746327.htm
Scraping (1008/1075): https://www.mudah.my/size-besar-full-loan-casa-villa-condominium-sg-chua-kajang-110238257.htm
Scra

Scraping (1065/1075): https://www.mudah.my/apartment-termurah-spring-ville-apartment-ukay-perdana-ampang-110041632.htm
Scraping (1066/1075): https://www.mudah.my/cheapest-and-biggest-unit-apartment-putera-permai-ampang-110045047.htm
Scraping (1067/1075): https://www.mudah.my/pangsapuri-sri-baiduri-ukay-perdana-ampang-110472211.htm
Scraping (1068/1075): https://www.mudah.my/pangsapuri-indah-projek-selangorku-full-loan-belakang-ipd-sepang-106500562.htm
Scraping (1069/1075): https://www.mudah.my/solstice-full-furnished-renovated-can-full-loan-cyberjaya-studio-mrt-110244047.htm
Scraping (1070/1075): https://www.mudah.my/studio-fully-furnished-cyberjaya-near-tamarind-kanvas-area-0-d-p-108426509.htm
Progress saved at 1070 listings
Scraping (1071/1075): https://www.mudah.my/booking-1k-kemensah-villa-saujana-melawati-ampang-1221sf-freehold-103708869.htm
Scraping (1072/1075): https://www.mudah.my/1k-booking-hijauan-puteri-condo-bandar-puteri-puchong-100-loan-108110456.htm
Scraping (1073/1075): 

### Selangor - House

In [14]:
# scrap selangor facilities
input_csv = "mudah_selangor_house_data.csv"
output_csv = "mudah_selangor_house_with_facilities.csv"
add_facilities_to_dataset(input_csv, output_csv, headless=True)


Scraping (1/645): https://www.mudah.my/15min-to-klia-gated-guarded-22x80-2-sty-townhouse-sepang-kota-warisan-109398999.htm
Scraping (2/645): https://www.mudah.my/view-klcc-bungalow-kg-bukit-sungai-puteh-bukit-belacan-ampang-111202135.htm
[Timeout] Attempt 1 for https://www.mudah.my/view-klcc-bungalow-kg-bukit-sungai-puteh-bukit-belacan-ampang-111202135.htm
[Timeout] Attempt 2 for https://www.mudah.my/view-klcc-bungalow-kg-bukit-sungai-puteh-bukit-belacan-ampang-111202135.htm
[Timeout] Attempt 3 for https://www.mudah.my/view-klcc-bungalow-kg-bukit-sungai-puteh-bukit-belacan-ampang-111202135.htm
Scraping (3/645): https://www.mudah.my/rumah-setingkat-taman-pinang-klang-renovated-111244560.htm
[Timeout] Attempt 1 for https://www.mudah.my/rumah-setingkat-taman-pinang-klang-renovated-111244560.htm
[Timeout] Attempt 2 for https://www.mudah.my/rumah-setingkat-taman-pinang-klang-renovated-111244560.htm
[Timeout] Attempt 3 for https://www.mudah.my/rumah-setingkat-taman-pinang-klang-renovated-111

[Timeout] Attempt 3 for https://www.mudah.my/freehold-below-market-value-2-storey-house-setia-impian-3-setia-alam-111285708.htm
Scraping (27/645): https://www.mudah.my/taman-sejati-sentosa-klang-3-storey-super-below-market-price-for-sale-111285701.htm
Scraping (28/645): https://www.mudah.my/facing-open-renovated-double-storey-superlink-lagenda-2-bukit-jeluton-110372740.htm
Scraping (29/645): https://www.mudah.my/booking-1k-double-storey-taman-pelangi-semenyih-1-109955842.htm
Scraping (30/645): https://www.mudah.my/klang-4xxk-freehold-double-storey-landed-111285741.htm
Progress saved at 30 listings
Scraping (31/645): https://www.mudah.my/2-storey-bungalow-seksyen-14-petaling-jaya-111285738.htm
[Timeout] Attempt 1 for https://www.mudah.my/2-storey-bungalow-seksyen-14-petaling-jaya-111285738.htm
[Timeout] Attempt 2 for https://www.mudah.my/2-storey-bungalow-seksyen-14-petaling-jaya-111285738.htm
[Timeout] Attempt 3 for https://www.mudah.my/2-storey-bungalow-seksyen-14-petaling-jaya-111285

[Timeout] Attempt 3 for https://www.mudah.my/for-sale-non-flood-tip-top-fully-reno-ext-mutiara-bukit-raja-klang-111285577.htm
Scraping (63/645): https://www.mudah.my/for-sale-mutiara-bukit-raja-klang-40x70-semi-d-fully-ext-reno-furnish-111285575.htm
[Timeout] Attempt 1 for https://www.mudah.my/for-sale-mutiara-bukit-raja-klang-40x70-semi-d-fully-ext-reno-furnish-111285575.htm
[Timeout] Attempt 2 for https://www.mudah.my/for-sale-mutiara-bukit-raja-klang-40x70-semi-d-fully-ext-reno-furnish-111285575.htm
[Timeout] Attempt 3 for https://www.mudah.my/for-sale-mutiara-bukit-raja-klang-40x70-semi-d-fully-ext-reno-furnish-111285575.htm
Scraping (64/645): https://www.mudah.my/for-sale-grandezza-eco-sanctuary-fully-reno-furnished-good-condition-111285574.htm
[Timeout] Attempt 1 for https://www.mudah.my/for-sale-grandezza-eco-sanctuary-fully-reno-furnished-good-condition-111285574.htm
[Timeout] Attempt 2 for https://www.mudah.my/for-sale-grandezza-eco-sanctuary-fully-reno-furnished-good-conditio

Scraping (100/645): https://www.mudah.my/full-loan-10k-cash-back-freehold-20x61-rantau-panjang-free-all-legal-94322975.htm
Progress saved at 100 listings
Scraping (101/645): https://www.mudah.my/pemaju-full-loan-rebat-47k-free-spa-jln-kebun-kemuning-rimbayu-sek-29-82123932.htm
Scraping (102/645): https://www.mudah.my/full-loan-free-all-legal-18x78-4r3b-freehold-meru-jln-iskandar-klang-108978614.htm
Scraping (103/645): https://www.mudah.my/full-loan-25k-cash-back-free-spa-mot-20x70-4r2b-freehold-bkt-kapar-wce-106369580.htm
Scraping (104/645): https://www.mudah.my/0-depo-full-loan-free-spa-mot-20x64-4r2b-1-tgt-baru-pulau-indah-klang-109513629.htm
Scraping (105/645): https://www.mudah.my/pemaju-10-off-0-depo-20x64-fully-built-kg-lombong-dkt-tmn-mas-puchong-105329153.htm
Scraping (106/645): https://www.mudah.my/deposit-flexible-nego-bare-unit-3-storey-bangi-avenue-3-seksyen-7-111236943.htm
Scraping (107/645): https://www.mudah.my/deposit-flexible-freehold-facing-open-2-storey-pinggiran-uka

[Timeout] Attempt 3 for https://www.mudah.my/endlot-double-storey-usj-4-subang-jaya-111285439.htm
Scraping (134/645): https://www.mudah.my/bandar-seri-coalfields-hampton-residence-full-reno-furnished-111285438.htm
Scraping (135/645): https://www.mudah.my/taman-ixora-bandar-baru-salak-tinggi-2-storey-terrace-reno-open-facing-111146795.htm
Scraping (136/645): https://www.mudah.my/masuk-cuci-kaki-2-storey-terrace-taman-kejora-sungai-chua-kajang-111260961.htm
[Timeout] Attempt 1 for https://www.mudah.my/masuk-cuci-kaki-2-storey-terrace-taman-kejora-sungai-chua-kajang-111260961.htm
[Timeout] Attempt 2 for https://www.mudah.my/masuk-cuci-kaki-2-storey-terrace-taman-kejora-sungai-chua-kajang-111260961.htm
[Timeout] Attempt 3 for https://www.mudah.my/masuk-cuci-kaki-2-storey-terrace-taman-kejora-sungai-chua-kajang-111260961.htm
Scraping (137/645): https://www.mudah.my/booking-1k-facing-open-2-storey-bangi-seksyen-4-tambahan-bangi-111285432.htm
[Timeout] Attempt 1 for https://www.mudah.my/booki

...

[Timeout] Attempt 1 for https://www.mudah.my/full-loan-new-2-storey-rawang-5-mins-to-tol-105268347.htm
[Timeout] Attempt 2 for https://www.mudah.my/full-loan-new-2-storey-rawang-5-mins-to-tol-105268347.htm
[Timeout] Attempt 3 for https://www.mudah.my/full-loan-new-2-storey-rawang-5-mins-to-tol-105268347.htm
Scraping (499/645): https://www.mudah.my/rawang-freehold-full-loan-cashback-24x65-new-near-highway-106451558.htm
[Timeout] Attempt 1 for https://www.mudah.my/rawang-freehold-full-loan-cashback-24x65-new-near-highway-106451558.htm
[Timeout] Attempt 2 for https://www.mudah.my/rawang-freehold-full-loan-cashback-24x65-new-near-highway-106451558.htm
[Timeout] Attempt 3 for https://www.mudah.my/rawang-freehold-full-loan-cashback-24x65-new-near-highway-106451558.htm
Scraping (500/645): https://www.mudah.my/0downpyt-corner-inter-laman-haris-eco-grandeur-bandar-puncak-alam-110785915.htm
Progress saved at 500 listings
Scraping (501/645): https://www.mudah.my/2sty-renovated-intermediate-house-

[Timeout] Attempt 2 for https://www.mudah.my/desa-mutiara-residence-town-house-renov-country-homes-rawang-sale-100286865.htm
[Timeout] Attempt 3 for https://www.mudah.my/desa-mutiara-residence-town-house-renov-country-homes-rawang-sale-100286865.htm
Scraping (533/645): https://www.mudah.my/2-storey-taman-desa-mas-desa-4-country-homes-rawang-for-sale-92802056.htm
[Timeout] Attempt 1 for https://www.mudah.my/2-storey-taman-desa-mas-desa-4-country-homes-rawang-for-sale-92802056.htm
[Timeout] Attempt 2 for https://www.mudah.my/2-storey-taman-desa-mas-desa-4-country-homes-rawang-for-sale-92802056.htm
[Timeout] Attempt 3 for https://www.mudah.my/2-storey-taman-desa-mas-desa-4-country-homes-rawang-for-sale-92802056.htm
Scraping (534/645): https://www.mudah.my/2-5sty-semi-d-suria-residen-40x80sft-batu-9-cheras-hilltop-taman-orkid-109515318.htm
Scraping (535/645): https://www.mudah.my/2-storey-terrace-house-7-min-to-gamuda-walk-mall-111282737.htm
[Timeout] Attempt 1 for https://www.mudah.my/2-s

Scraping (565/645): https://www.mudah.my/2-storey-link-house-for-sale-setia-permai-2-setia-alam-109715971.htm
Scraping (566/645): https://www.mudah.my/2storey-terrace-house-taman-tun-perak-rawang-for-sale-104495921.htm
Scraping (567/645): https://www.mudah.my/3-storey-terraced-house-dolomite-templer-for-sale-105110056.htm
Scraping (568/645): https://www.mudah.my/2-storey-end-lot-22-x-70-fully-extend-bandar-tasik-kesuma-semenyih-107206675.htm
Scraping (569/645): https://www.mudah.my/golf-course-view-luas-2-5-storey-bungalow-bangi-golf-resort-bangi-110200022.htm
Scraping (570/645): https://www.mudah.my/corner-big-land-end-road-2-storey-bandar-tun-hussein-onn-cheras-110170438.htm
Progress saved at 570 listings
Scraping (571/645): https://www.mudah.my/freehold-furnish-2-5-storey-semi-d-kayu-manis-garden-villas-bangi-109379580.htm
Scraping (572/645): https://www.mudah.my/freehold-2-storey-terrace-house-bandar-sungai-long-111282530.htm
[Timeout] Attempt 1 for https://www.mudah.my/freehold-2-

[Timeout] Attempt 2 for https://www.mudah.my/corner-lot-2-storey-terrace-house-taman-sri-putra-2-sungai-buloh-110551424.htm
[Timeout] Attempt 3 for https://www.mudah.my/corner-lot-2-storey-terrace-house-taman-sri-putra-2-sungai-buloh-110551424.htm
Scraping (597/645): https://www.mudah.my/renovated-2-storey-terrace-house-setia-permai-1-u13-setia-alam-110551422.htm
[Timeout] Attempt 1 for https://www.mudah.my/renovated-2-storey-terrace-house-setia-permai-1-u13-setia-alam-110551422.htm
[Timeout] Attempt 2 for https://www.mudah.my/renovated-2-storey-terrace-house-setia-permai-1-u13-setia-alam-110551422.htm
[Timeout] Attempt 3 for https://www.mudah.my/renovated-2-storey-terrace-house-setia-permai-1-u13-setia-alam-110551422.htm
Scraping (598/645): https://www.mudah.my/facing-open-end-lot-2-storey-terrace-house-seksyen-6-shah-alam-110551421.htm
[Timeout] Attempt 1 for https://www.mudah.my/facing-open-end-lot-2-storey-terrace-house-seksyen-6-shah-alam-110551421.htm
[Timeout] Attempt 2 for http

[Timeout] Attempt 3 for https://www.mudah.my/under-market-value-2-5-sty-terrace-bukit-serdang-sek-14-111282228.htm
Scraping (633/645): https://www.mudah.my/kajang-taman-bukit-mewah-freehold-22-x-75-near-mrt-108777272.htm
[Timeout] Attempt 1 for https://www.mudah.my/kajang-taman-bukit-mewah-freehold-22-x-75-near-mrt-108777272.htm
[Timeout] Attempt 2 for https://www.mudah.my/kajang-taman-bukit-mewah-freehold-22-x-75-near-mrt-108777272.htm
[Timeout] Attempt 3 for https://www.mudah.my/kajang-taman-bukit-mewah-freehold-22-x-75-near-mrt-108777272.htm
Scraping (634/645): https://www.mudah.my/2-storey-corner-lot-3960-sf-freehold-below-market-setia-ecohill-2-109297927.htm
Scraping (635/645): https://www.mudah.my/2-storey-damansara-utama-23-x-75-freehold-ss21-ss-21-pj-103958777.htm
Scraping (636/645): https://www.mudah.my/2-5sty-house-for-sale-ss12-ss-12-subang-jaya-ss-15-below-market-price-110752304.htm
[Timeout] Attempt 1 for https://www.mudah.my/2-5sty-house-for-sale-ss12-ss-12-subang-jaya-ss

### KL - Apartment & Condominium

In [2]:
# scrap KL facilities
input_csv = "mudah_kl_apartment_condominium_data.csv"
output_csv = "mudah_kl_apartment_condominium_with_facilities.csv"
add_facilities_to_dataset(input_csv, output_csv, headless=True)


Scraping (1/645): https://www.mudah.my/level-3-bayu-tasik-condo-110487381.htm
Scraping (2/645): https://www.mudah.my/bukit-jalil-duplex-penthouse-sky-bungalow-garden-in-the-sky-110737394.htm
Scraping (3/645): https://www.mudah.my/for-sale-vista-tasik-condominium-bandar-sri-permaisuri-cheras-kl-111237963.htm
Scraping (4/645): https://www.mudah.my/sri-kuching-apartment-for-sale-at-jalan-kuching-kl-111211912.htm
Scraping (5/645): https://www.mudah.my/chic-studio-ceylonz-suites-bukit-ceylon-kuala-lumpur-for-sale-110720544.htm
Scraping (6/645): https://www.mudah.my/cheapest-1025sqft-mentari-condominium-bandar-permaisuri-cheras-kl-110884553.htm
Scraping (7/645): https://www.mudah.my/2-parking-endah-puri-condominium-sri-petaling-kuala-lumpur-111272898.htm
Scraping (8/645): https://www.mudah.my/ascenda-residensi-condo-setapak-kuala-lumpur-110843729.htm
Scraping (9/645): https://www.mudah.my/corner-unit-flat-seksyen-4-wangsa-maju-tingkat-1-non-bumi-unit-110806282.htm
Scraping (10/645): https://

[Timeout] Attempt 1 for https://www.mudah.my/1200sf-3-rooms-sentul-balcony-freehold-20-mins-to-pj-mon-t-kiara-110994945.htm
[Timeout] Attempt 2 for https://www.mudah.my/1200sf-3-rooms-sentul-balcony-freehold-20-mins-to-pj-mon-t-kiara-110994945.htm
[Timeout] Attempt 3 for https://www.mudah.my/1200sf-3-rooms-sentul-balcony-freehold-20-mins-to-pj-mon-t-kiara-110994945.htm
Scraping (39/645): https://www.mudah.my/serini-melawati-100-loan-0-deposit-111272397.htm
Scraping (40/645): https://www.mudah.my/freehold-fully-renovated-flat-taman-melati-facing-open-with-balcony-108183410.htm
Progress saved at 40 listings
Scraping (41/645): https://www.mudah.my/nice-freehold-open-title-apartment-intan-setiawangsa-hot-location-110532740.htm
Scraping (42/645): https://www.mudah.my/kami-mont-kiara-residence-109012911.htm
Scraping (43/645): https://www.mudah.my/idaman-residence-kl-city-centre-klcc-90040200.htm
Scraping (44/645): https://www.mudah.my/1200sf-3-rooms-sentul-balcony-freehold-20-mins-to-pj-mon-

Scraping (96/645): https://www.mudah.my/renovated-flat-mutiara-magna-kepong-111271190.htm
Scraping (97/645): https://www.mudah.my/the-era-duta-north-segambut-2-bedrooms-unit-111271189.htm
Scraping (98/645): https://www.mudah.my/bukit-jalil-studio-500sf-ready-move-in-fully-furnish-walk-mrt-shop-111052317.htm
[Timeout] Attempt 1 for https://www.mudah.my/bukit-jalil-studio-500sf-ready-move-in-fully-furnish-walk-mrt-shop-111052317.htm
[Timeout] Attempt 2 for https://www.mudah.my/bukit-jalil-studio-500sf-ready-move-in-fully-furnish-walk-mrt-shop-111052317.htm
[Timeout] Attempt 3 for https://www.mudah.my/bukit-jalil-studio-500sf-ready-move-in-fully-furnish-walk-mrt-shop-111052317.htm
Scraping (99/645): https://www.mudah.my/klcc-studio-230k-ready-move-in-free-furnished-beside-mrt-mall-111128716.htm
[Timeout] Attempt 1 for https://www.mudah.my/klcc-studio-230k-ready-move-in-free-furnished-beside-mrt-mall-111128716.htm
[Timeout] Attempt 2 for https://www.mudah.my/klcc-studio-230k-ready-move-in-

Scraping (152/645): https://www.mudah.my/exclusive-fully-furnished-unit-for-sale-at-rica-residence-sentul-111269505.htm
Scraping (153/645): https://www.mudah.my/100-loan-the-leafz-condo-1025-sqf-kuchai-lama-salak-south-garden-110362847.htm
Scraping (154/645): https://www.mudah.my/fully-reno-perdana-puri-apartment-100-loan-desa-aman-puri-kepong-110362947.htm
Scraping (155/645): https://www.mudah.my/villa-orkid-condominium-bukit-prima-pelangi-segambut-dalam-for-sale-110496295.htm
Scraping (156/645): https://www.mudah.my/blok-f-pangsapuri-bandar-tasik-selatan-fully-renovated-sungai-besi-97750618.htm
Scraping (157/645): https://www.mudah.my/flat-cheras-ria-freehold-100-loan-fully-reno-taman-cheras-utama-104297638.htm
Scraping (158/645): https://www.mudah.my/residensi-pano-batu-3-2-bedrooms-unit-111269599.htm
Scraping (159/645): https://www.mudah.my/iris-apartment-fully-reno-100-loan-rumah-corner-taman-desa-107273072.htm
Scraping (160/645): https://www.mudah.my/cheapest-nice-reno-fully-furn

Scraping (224/645): https://www.mudah.my/cash-back-ketumbar-heights-903sf-cheras-utama-few-unit-available-111268511.htm
Scraping (225/645): https://www.mudah.my/renovated-flat-bandar-baru-sentul-kl-near-to-lrt-111268330.htm
Scraping (226/645): https://www.mudah.my/termudah-m3-residency-taman-melati-kuala-lumpur-108165566.htm
Scraping (227/645): https://www.mudah.my/corner-the-holmes-2-bandar-tun-razak-near-hukm-111268474.htm
Scraping (228/645): https://www.mudah.my/setapak-freehold-condo-walking-distance-mrt-speed-ramp-carpark-110693950.htm
[Timeout] Attempt 1 for https://www.mudah.my/setapak-freehold-condo-walking-distance-mrt-speed-ramp-carpark-110693950.htm
[Timeout] Attempt 2 for https://www.mudah.my/setapak-freehold-condo-walking-distance-mrt-speed-ramp-carpark-110693950.htm
[Timeout] Attempt 3 for https://www.mudah.my/setapak-freehold-condo-walking-distance-mrt-speed-ramp-carpark-110693950.htm
Scraping (229/645): https://www.mudah.my/paling-murah-100-loan-the-holmes-1-condo-banda

...

Scraping (354/645): https://www.mudah.my/0-downpayment-klcc-hotel-residence-grr-15-60-meter-to-mrt-111038914.htm
Scraping (355/645): https://www.mudah.my/60m-to-mrt-hotel-investment-special-layout-fully-management-110593381.htm
Scraping (356/645): https://www.mudah.my/low-density-condo-sentul-matured-neighbourhood-privacy-111193966.htm
Scraping (357/645): https://www.mudah.my/duplex-designer-condo-in-bukit-jalil-near-pavilion-mall-111192048.htm
Scraping (358/645): https://www.mudah.my/freehold-condominium-setapak-ria-911sqft-100-loan-offer-111171584.htm
Scraping (359/645): https://www.mudah.my/residensi-alam-damai-kuala-lumpur-cheras-950sqft-2-parking-110838056.htm
Scraping (360/645): https://www.mudah.my/flat-desa-petaling-559sf-2r1b-partly-furnished-nice-unit-107434867.htm
Progress saved at 360 listings
Scraping (361/645): https://www.mudah.my/the-rainz-bukit-jalil-6-min-to-pavilion-bukit-jalil-111265127.htm
Scraping (362/645): https://www.mudah.my/100-loan-zetapark-studio-central-ma

Scraping (422/645): https://www.mudah.my/setiawangsa-new-luxury-condo-110146514.htm
Scraping (423/645): https://www.mudah.my/nice-unit-fully-furnished-bintang-mas-condominium-near-lrt-cheras-111263776.htm
Scraping (424/645): https://www.mudah.my/ready-move-in-near-mrt-equine-mall-3-4-room-0-downpayment-big-yard-111188585.htm
Scraping (425/645): https://www.mudah.my/pangsapuri-teratai-mewah-apartment-670sf-setapak-jalan-langkawi-1kdepo-110802131.htm
Scraping (426/645): https://www.mudah.my/riana-green-east-condo-wangsa-maju-near-lrt-move-in-condition-109827981.htm
Scraping (427/645): https://www.mudah.my/sinaran-wangsa-maju-opposite-aeon-jusco-near-lrt-wangsa-maju-105121278.htm
Scraping (428/645): https://www.mudah.my/danau-murni-condo-below-market-value-renovated-taman-desa-kl-111263998.htm
Scraping (429/645): https://www.mudah.my/wangsa-maju-seasons-garden-1-car-park-near-aminities-110603960.htm
Scraping (430/645): https://www.mudah.my/seri-riana-condo-move-in-condition-near-wangsa-wa

Progress saved at 490 listings
Scraping (491/645): https://www.mudah.my/south-view-fulll-furnish-condo-corner-unit-at-bangsar-south-110294463.htm
Scraping (492/645): https://www.mudah.my/pv-13-platinum-lake-condominium-setapak-kuala-lumpur-111181326.htm
Scraping (493/645): https://www.mudah.my/sri-intan-2-freehold-jalan-ipoh-kuala-lumpur-110810647.htm
Scraping (494/645): https://www.mudah.my/kl-traders-square-residences-setapak-for-sale-end-lot-111221608.htm
Scraping (495/645): https://www.mudah.my/prisma-cheras-taman-midah-condo-for-sale-111235053.htm
Scraping (496/645): https://www.mudah.my/full-loan-dekat-lrt-angsana-apartment-pandan-indah-110988155.htm
[Timeout] Attempt 1 for https://www.mudah.my/full-loan-dekat-lrt-angsana-apartment-pandan-indah-110988155.htm
[Timeout] Attempt 2 for https://www.mudah.my/full-loan-dekat-lrt-angsana-apartment-pandan-indah-110988155.htm
[Timeout] Attempt 3 for https://www.mudah.my/full-loan-dekat-lrt-angsana-apartment-pandan-indah-110988155.htm
Scrap

Scraping (560/645): https://www.mudah.my/seasons-garden-residence-wangsa-maju-kl-110834566.htm
Progress saved at 560 listings
Scraping (561/645): https://www.mudah.my/pv2-setapak-melati-renovated-2-carparks-high-floor-for-sale-111245502.htm
Scraping (562/645): https://www.mudah.my/50m-to-klcc-klcc-investment-condo-fully-furnished-monthly-1-8k-108038067.htm
Scraping (563/645): https://www.mudah.my/residensi-bukit-jalil-taman-ltat-fully-furnished-90-units-only-111206828.htm
Scraping (564/645): https://www.mudah.my/happy-garden-one-mesraria-apartment-for-sale-111260572.htm
Scraping (565/645): https://www.mudah.my/idaman-putera-condominium-setapak-low-floor-built-up-1335sqft-111260716.htm
Scraping (566/645): https://www.mudah.my/freehold-low-floor-ketumbar-heights-prime-loc-100-loan-0-deposit-111260700.htm
Scraping (567/645): https://www.mudah.my/pandan-jaya-flat-block-l13-100-loan-111260685.htm
[Timeout] Attempt 1 for https://www.mudah.my/pandan-jaya-flat-block-l13-100-loan-111260685.htm


Scraping (618/645): https://www.mudah.my/corner-unit-luas-condo-razak-city-rc-residence-sg-besi-kuala-lumpur-105576973.htm
Scraping (619/645): https://www.mudah.my/below-market-the-pano-jalan-ipoh-near-birch-rica-court-111259885.htm
[Timeout] Attempt 1 for https://www.mudah.my/below-market-the-pano-jalan-ipoh-near-birch-rica-court-111259885.htm
[Timeout] Attempt 2 for https://www.mudah.my/below-market-the-pano-jalan-ipoh-near-birch-rica-court-111259885.htm
[Timeout] Attempt 3 for https://www.mudah.my/below-market-the-pano-jalan-ipoh-near-birch-rica-court-111259885.htm
Scraping (620/645): https://www.mudah.my/grr-roi-up-to-15-2br-soho-freehold-8-mins-to-klcc-monthly-rm1800-111132802.htm
Progress saved at 620 listings
Scraping (621/645): https://www.mudah.my/freehold-klcc-roi-15-lifetime-free-instalment-free-maintenance-111132881.htm
Scraping (622/645): https://www.mudah.my/duplex-711sf-8th-stellar-furnished-sri-petaling-kl-110607304.htm
Scraping (623/645): https://www.mudah.my/100loan-f

### KL - House

In [2]:
# scrap KL facilities
input_csv = "mudah_kl_house_data.csv"
output_csv = "mudah_kl_house_with_facilities.csv"
add_facilities_to_dataset(input_csv, output_csv, headless=True)


Scraping (1/645): https://www.mudah.my/taman-ehsan-kepong-corner-lot-newly-renovated-fully-extended-k-l-111271771.htm
Scraping (2/645): https://www.mudah.my/rumah-mewah-3-tingkat-eksklusif-gombak-kl-111242241.htm
[Timeout] Attempt 1 for https://www.mudah.my/rumah-mewah-3-tingkat-eksklusif-gombak-kl-111242241.htm
[Timeout] Attempt 2 for https://www.mudah.my/rumah-mewah-3-tingkat-eksklusif-gombak-kl-111242241.htm
[Timeout] Attempt 3 for https://www.mudah.my/rumah-mewah-3-tingkat-eksklusif-gombak-kl-111242241.htm
Scraping (3/645): https://www.mudah.my/taman-bukit-piatu-mutiara-bungalow-melaka-town-near-mall-and-hospital-111202414.htm
[Timeout] Attempt 1 for https://www.mudah.my/taman-bukit-piatu-mutiara-bungalow-melaka-town-near-mall-and-hospital-111202414.htm
[Timeout] Attempt 2 for https://www.mudah.my/taman-bukit-piatu-mutiara-bungalow-melaka-town-near-mall-and-hospital-111202414.htm
[Timeout] Attempt 3 for https://www.mudah.my/taman-bukit-piatu-mutiara-bungalow-melaka-town-near-mall-a

[Timeout] Attempt 3 for https://www.mudah.my/2-sty-terrace-cheras-utama-111283300.htm
Progress saved at 30 listings
Scraping (31/645): https://www.mudah.my/1-storey-house-taman-maju-jaya-reno-22x70-100-loan-pandan-indah-108725824.htm
[Timeout] Attempt 1 for https://www.mudah.my/1-storey-house-taman-maju-jaya-reno-22x70-100-loan-pandan-indah-108725824.htm
[Timeout] Attempt 2 for https://www.mudah.my/1-storey-house-taman-maju-jaya-reno-22x70-100-loan-pandan-indah-108725824.htm
[Timeout] Attempt 3 for https://www.mudah.my/1-storey-house-taman-maju-jaya-reno-22x70-100-loan-pandan-indah-108725824.htm
Scraping (32/645): https://www.mudah.my/renovated-extend-2-storey-terrace-house-wangsa-ceria-melawati-kl-111283262.htm
[Timeout] Attempt 1 for https://www.mudah.my/renovated-extend-2-storey-terrace-house-wangsa-ceria-melawati-kl-111283262.htm
[Timeout] Attempt 2 for https://www.mudah.my/renovated-extend-2-storey-terrace-house-wangsa-ceria-melawati-kl-111283262.htm
[Timeout] Attempt 3 for https:

Scraping (57/645): https://www.mudah.my/murah-2-storey-terrace-taman-keramat-permai-kuala-lumpur-109104189.htm
[Timeout] Attempt 1 for https://www.mudah.my/murah-2-storey-terrace-taman-keramat-permai-kuala-lumpur-109104189.htm
[Timeout] Attempt 2 for https://www.mudah.my/murah-2-storey-terrace-taman-keramat-permai-kuala-lumpur-109104189.htm
[Timeout] Attempt 3 for https://www.mudah.my/murah-2-storey-terrace-taman-keramat-permai-kuala-lumpur-109104189.htm
Scraping (58/645): https://www.mudah.my/cheras-double-storey-bungalow-taman-ferngrove-near-taman-cuepacs-110383862.htm
[Timeout] Attempt 1 for https://www.mudah.my/cheras-double-storey-bungalow-taman-ferngrove-near-taman-cuepacs-110383862.htm
[Timeout] Attempt 2 for https://www.mudah.my/cheras-double-storey-bungalow-taman-ferngrove-near-taman-cuepacs-110383862.htm
[Timeout] Attempt 3 for https://www.mudah.my/cheras-double-storey-bungalow-taman-ferngrove-near-taman-cuepacs-110383862.htm
Scraping (59/645): https://www.mudah.my/freehold-t

[Timeout] Attempt 3 for https://www.mudah.my/taman-melawati-2-sty-house-jalan-a-22x80-freehold-111279640.htm
Scraping (87/645): https://www.mudah.my/corner-unit-ground-floor-townhouse-taman-setapak-indah-kl-111132630.htm
[Timeout] Attempt 1 for https://www.mudah.my/corner-unit-ground-floor-townhouse-taman-setapak-indah-kl-111132630.htm
[Timeout] Attempt 2 for https://www.mudah.my/corner-unit-ground-floor-townhouse-taman-setapak-indah-kl-111132630.htm
[Timeout] Attempt 3 for https://www.mudah.my/corner-unit-ground-floor-townhouse-taman-setapak-indah-kl-111132630.htm
Scraping (88/645): https://www.mudah.my/taman-ehsan-kepong-corner-lot-newly-renovated-fully-extended-k-l-111271771.htm
Scraping (89/645): https://www.mudah.my/booking-1k-ground-floor-facing-open-taman-melati-townhouse-setapak-111040402.htm
[Timeout] Attempt 1 for https://www.mudah.my/booking-1k-ground-floor-facing-open-taman-melati-townhouse-setapak-111040402.htm
[Timeout] Attempt 2 for https://www.mudah.my/booking-1k-ground

Scraping (113/645): https://www.mudah.my/best-price-strategic-double-storey-house-taman-beringin-kepong-111015245.htm
Scraping (114/645): https://www.mudah.my/brand-new-unit-areca-residence-semi-d-kepong-36x80-105799627.htm
Scraping (115/645): https://www.mudah.my/corner-gated-guarded-big-taman-melati-setapak-3-storey-bungalow-109372293.htm
Scraping (116/645): https://www.mudah.my/single-storey-bungalow-taman-sri-delima-jalan-kuching-segambut-kl-111278283.htm
Scraping (117/645): https://www.mudah.my/booking-1k-semi-d-cluster-putra-perdana-nego-diskaun-100k-non-bumi-109982815.htm
Scraping (118/645): https://www.mudah.my/last-13-unit-bungalow-individual-title-freehold-40-x-80-kuala-lumpur-111191265.htm
Scraping (119/645): https://www.mudah.my/gombak-height-bungalow-individual-title-freehold-40x-80-kuala-lumpur-111253719.htm
Scraping (120/645): https://www.mudah.my/fantastic-facing-open-double-storey-terrace-alam-damai-cheras-111155364.htm
Progress saved at 120 listings
Scraping (121/645)

...

Scraping (519/645): https://www.mudah.my/taman-bukit-piatu-mutiara-bungalow-melaka-town-near-mall-and-hospital-111202414.htm
[Timeout] Attempt 1 for https://www.mudah.my/taman-bukit-piatu-mutiara-bungalow-melaka-town-near-mall-and-hospital-111202414.htm
[Timeout] Attempt 2 for https://www.mudah.my/taman-bukit-piatu-mutiara-bungalow-melaka-town-near-mall-and-hospital-111202414.htm
[Timeout] Attempt 3 for https://www.mudah.my/taman-bukit-piatu-mutiara-bungalow-melaka-town-near-mall-and-hospital-111202414.htm
Scraping (520/645): https://www.mudah.my/stunning-2-storey-fully-renovated-home-in-taman-taynton-view-111244911.htm
[Timeout] Attempt 1 for https://www.mudah.my/stunning-2-storey-fully-renovated-home-in-taman-taynton-view-111244911.htm
[Timeout] Attempt 2 for https://www.mudah.my/stunning-2-storey-fully-renovated-home-in-taman-taynton-view-111244911.htm
[Timeout] Attempt 3 for https://www.mudah.my/stunning-2-storey-fully-renovated-home-in-taman-taynton-view-111244911.htm
Progress sav

[Timeout] Attempt 2 for https://www.mudah.my/termurah-freehold-double-storey-terrace-wangsa-murni-wangsa-melawati-109945109.htm
[Timeout] Attempt 3 for https://www.mudah.my/termurah-freehold-double-storey-terrace-wangsa-murni-wangsa-melawati-109945109.htm
Scraping (548/645): https://www.mudah.my/brand-new-3-storey-bungalow-semi-d-gombak-heights-near-selayang-111031878.htm
Scraping (549/645): https://www.mudah.my/brand-new-3-storey-bungalow-semi-d-gombak-height-near-selayang-111031452.htm
Scraping (550/645): https://www.mudah.my/partially-furnished-bungalow-anjung-melati-3-for-sale-109228056.htm
Progress saved at 550 listings
Scraping (551/645): https://www.mudah.my/cantik-renovated-single-storey-lorong-jujur-bandar-tun-razak-cheras-110765018.htm
[Timeout] Attempt 1 for https://www.mudah.my/cantik-renovated-single-storey-lorong-jujur-bandar-tun-razak-cheras-110765018.htm
[Timeout] Attempt 2 for https://www.mudah.my/cantik-renovated-single-storey-lorong-jujur-bandar-tun-razak-cheras-1107

[Timeout] Attempt 2 for https://www.mudah.my/most-wanted-renovated-2-storey-taman-setiawangsa-110668822.htm
[Timeout] Attempt 3 for https://www.mudah.my/most-wanted-renovated-2-storey-taman-setiawangsa-110668822.htm
Scraping (574/645): https://www.mudah.my/booking-rm1000-double-storey-intermediate-au-1-keramat-kl-111240320.htm
[Timeout] Attempt 1 for https://www.mudah.my/booking-rm1000-double-storey-intermediate-au-1-keramat-kl-111240320.htm
[Timeout] Attempt 2 for https://www.mudah.my/booking-rm1000-double-storey-intermediate-au-1-keramat-kl-111240320.htm
[Timeout] Attempt 3 for https://www.mudah.my/booking-rm1000-double-storey-intermediate-au-1-keramat-kl-111240320.htm
Scraping (575/645): https://www.mudah.my/double-storey-terrace-taman-permata-au5-keramat-kuala-lumpur-111240103.htm
[Timeout] Attempt 1 for https://www.mudah.my/double-storey-terrace-taman-permata-au5-keramat-kuala-lumpur-111240103.htm
[Timeout] Attempt 2 for https://www.mudah.my/double-storey-terrace-taman-permata-au5

[Timeout] Attempt 3 for https://www.mudah.my/country-height-kajang-110164711.htm
Scraping (603/645): https://www.mudah.my/bukit-jalil-end-lot-2-5-storey-house-kuala-lumpur-107801880.htm
Scraping (604/645): https://www.mudah.my/2-sty-house-pantai-permai-pantai-dalam-kuala-lumpur-109474405.htm
[Timeout] Attempt 1 for https://www.mudah.my/2-sty-house-pantai-permai-pantai-dalam-kuala-lumpur-109474405.htm
[Timeout] Attempt 2 for https://www.mudah.my/2-sty-house-pantai-permai-pantai-dalam-kuala-lumpur-109474405.htm
[Timeout] Attempt 3 for https://www.mudah.my/2-sty-house-pantai-permai-pantai-dalam-kuala-lumpur-109474405.htm
Scraping (605/645): https://www.mudah.my/freehold-termurah-double-storey-terrace-house-taman-setapak-kl-111281786.htm
[Timeout] Attempt 1 for https://www.mudah.my/freehold-termurah-double-storey-terrace-house-taman-setapak-kl-111281786.htm
[Timeout] Attempt 2 for https://www.mudah.my/freehold-termurah-double-storey-terrace-house-taman-setapak-kl-111281786.htm
[Timeout] At

Scraping (630/645): https://www.mudah.my/renovated-double-storey-terrace-jalan-makmur-bandar-tun-razak-cheras-111234019.htm
[Timeout] Attempt 1 for https://www.mudah.my/renovated-double-storey-terrace-jalan-makmur-bandar-tun-razak-cheras-111234019.htm
[Timeout] Attempt 2 for https://www.mudah.my/renovated-double-storey-terrace-jalan-makmur-bandar-tun-razak-cheras-111234019.htm
[Timeout] Attempt 3 for https://www.mudah.my/renovated-double-storey-terrace-jalan-makmur-bandar-tun-razak-cheras-111234019.htm
Progress saved at 630 listings
Scraping (631/645): https://www.mudah.my/renovated-extended-2-5-storey-terrace-bandar-sri-damansara-sd-7-111233974.htm
[Timeout] Attempt 1 for https://www.mudah.my/renovated-extended-2-5-storey-terrace-bandar-sri-damansara-sd-7-111233974.htm
[Timeout] Attempt 2 for https://www.mudah.my/renovated-extended-2-5-storey-terrace-bandar-sri-damansara-sd-7-111233974.htm
[Timeout] Attempt 3 for https://www.mudah.my/renovated-extended-2-5-storey-terrace-bandar-sri-da

## Data Overview

In [5]:
import pandas as pd

# Load the dataset 
df_mudah1 = pd.read_csv('mudah_selangor_apartment_condominium_with_facilities.csv')
df_mudah2 = pd.read_csv('mudah_kl_apartment_condominium_with_facilities.csv')
df_mudah3 = pd.read_csv('mudah_selangor_house_with_facilities.csv')
df_mudah4 = pd.read_csv('mudah_kl_house_with_facilities.csv')

# Display the shape 
print("df_mudah1 shape:", df_mudah1.shape)
print("df_mudah2 shape:", df_mudah2.shape)
print("df_mudah3 shape:", df_mudah3.shape)
print("df_mudah4 shape:", df_mudah4.shape)

df_mudah1.head()

df_mudah1 shape: (1075, 9)
df_mudah2 shape: (645, 9)
df_mudah3 shape: (645, 9)
df_mudah4 shape: (645, 9)


,Price,Location,Posted,Size (sqft),Bedrooms,Bathrooms,Tenure,URL,Facilities
0,"RM 140,000","Taman Kajang Mewah, Kajang","Apartment For Sale, listed 2 days ago",850,3,2,Freehold,https://www.mudah.my/pangsapuri-taman-kajang-m...,Parking
1,"RM 365,000","Pacific Place, Ara Damansara","Service Residence For Sale, listed 18 hours ago",697,2,1,Leasehold,https://www.mudah.my/pj-ara-damansara-next-lrt...,"Parking, Security, Lift, Swimming Pool, Playgr..."
2,"RM 330,000","Residensi Zamrud @ Kajang, Kajang","Condominium For Sale, listed 4 hours ago","1,045",3,2,Freehold,https://www.mudah.my/residensi-zamrud-kajang-r...,"Parking, Security, Lift, Swimming Pool, Playgr..."
3,"RM 230,000","D'Cassia Apartment, Semenyih","Apartment For Sale, listed 11 minutes ago",850,3,2,Freehold,https://www.mudah.my/100-loan-d-cassia-apartme...,"Parking, Security, Lift, Swimming Pool, Playgr..."
4,"RM 290,000","Vesta View Apartment @ Taman Putra Impian, Kajang","Apartment For Sale, listed 11 minutes ago","1,044",3,3,Leasehold,https://www.mudah.my/100-loan-vesta-view-apart...,"Parking, Security, Lift, Swimming Pool, Playgr..."


In [6]:
# Combine the datasets
df_mudah = pd.concat([df_mudah1, df_mudah2, df_mudah3, df_mudah4], ignore_index=True)

# Display the shape
print("Mudah dataset shape:", df_mudah.shape)

# Save to a new file
df_mudah.to_csv("mudah_data.csv", index=False)

Mudah dataset shape: (3010, 9)


# PropertyGuru

## Scrap General Info

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import random
import os

def scrape_batch(start_page, end_page, output_csv, resume=False):
    data = []
    seen = set()
    
    # Resume functionality - only check for existing URLs
    if resume and os.path.exists(output_csv):
        try:
            existing_df = pd.read_csv(output_csv)
            seen = set(existing_df['URL'].dropna().unique())
            print(f"Resuming with {len(seen)} existing URLs")
        except:
            pass

    for page in range(start_page, end_page + 1):
        driver = webdriver.Chrome()
        driver.execute_cdp_cmd('Network.setUserAgentOverride', {
            "userAgent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36"
        })
        
        try:
            print(f"Scraping page {page}...")
            url = get_url(page)
            driver.get(url)
            time.sleep(random.uniform(5, 7))

            listings = driver.find_elements(By.XPATH, '//div[contains(@class, "listing-card")]')

            for listing in listings:
                try:
                    url = listing.find_element(By.XPATH, './/a').get_attribute('href')
                except NoSuchElementException:
                    url = ''

                if not url or url in seen:
                    continue
                seen.add(url)

                try:
                    price = listing.find_element(By.XPATH, './/div[contains(@class, "price")]').text
                except NoSuchElementException:
                    price = ''

                try:
                    location = listing.find_element(By.XPATH, './/div[contains(@class, "address")]').text
                except NoSuchElementException:
                    location = ''

                try:
                    posted = listing.find_element(
                        By.XPATH, './/li[@class="info-item"][.//img[@alt="clock-circle-o"]]/span'
                    ).text
                except NoSuchElementException:
                    posted = ''

                try:
                    bedrooms = listing.find_element(
                        By.XPATH, './/li[@class="info-item"][.//img[@alt="bed-o"]]/span'
                    ).text
                except NoSuchElementException:
                    bedrooms = ''

                try:
                    bathrooms = listing.find_element(
                        By.XPATH, './/li[@class="info-item"][.//img[@alt="bath-o"]]/span'
                    ).text
                except NoSuchElementException:
                    bathrooms = ''

                try:
                    size = listing.find_element(
                        By.XPATH, './/span[contains(@class,"info-value") and contains(text(),"sqft")]'
                    ).text.strip()
                except NoSuchElementException:
                    size = ''

                try:
                    badges = listing.find_elements(By.XPATH, './/span[@da-id="lc-info-badge"]')
                    tenure = ''
                    property_type = ''
                    if badges:
                        property_type = badges[0].text.strip()
                        for badge in badges:
                            text = badge.text.strip()
                            if "freehold" in text.lower() or "leasehold" in text.lower():
                                tenure = text
                except NoSuchElementException:
                    property_type = ''
                    tenure = ''

                data.append({
                    "Price": price,
                    "Location": location,
                    "Posted": posted,
                    "Size (sqft)": size,
                    "Bedrooms": bedrooms,
                    "Bathrooms": bathrooms,
                    "Property Type": property_type,
                    "Tenure": tenure,
                    "URL": url
                })

        except Exception as e:
            print(f"Error on page {page}: {str(e)}")
        finally:
            driver.quit()
            time.sleep(random.uniform(3, 5))

    # Save CSV (with append mode if resuming)
    if data:
        df = pd.DataFrame(data)
        mode = 'a' if resume and os.path.exists(output_csv) else 'w'
        header = not (resume and os.path.exists(output_csv))
        df.to_csv(output_csv, mode=mode, header=header, index=False)
        print(f"✅ Saved batch {start_page}-{end_page} to '{output_csv}'")


### Selangor - Apartment & Condominium

In [29]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "_freetextDisplay": "Selangor",
        "isCommercial": "false",
        "listingType": "sale",
        "page": "1",  # Static page parameter
        "propertyTypeCode": ["APT"],
        "propertyTypeGroup": "N",
        "regionCode": "45nk1"  # Selangor region code
    }
    # Build the URL with the pattern
    return f"{base_url}/{page}?_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&listingType={params['listingType']}&page=1&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]]) + \
           f"&propertyTypeGroup={params['propertyTypeGroup']}&regionCode={params['regionCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_selangor_apartment_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_selangor_apartment_data.csv'


In [30]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "_freetextDisplay": "Selangor",
        "isCommercial": "false",
        "listingType": "sale",
        "page": "1",  # Static page parameter
        "propertyTypeCode": ["CONDO"],
        "propertyTypeGroup": "N",
        "regionCode": "45nk1"  # Selangor region code
    }
    # Build the URL with the pattern
    return f"{base_url}/{page}?_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&listingType={params['listingType']}&page=1&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]]) + \
           f"&propertyTypeGroup={params['propertyTypeGroup']}&regionCode={params['regionCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_selangor_condominium_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_selangor_condominium_data.csv'


In [31]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "_freetextDisplay": "Selangor",
        "isCommercial": "false",
        "listingType": "sale",
        "page": "1",  # Static page parameter
        "propertyTypeCode": ["FLAT"],
        "propertyTypeGroup": "N",
        "regionCode": "45nk1"  # Selangor region code
    }
    # Build the URL with the pattern
    return f"{base_url}/{page}?_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&listingType={params['listingType']}&page=1&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]]) + \
           f"&propertyTypeGroup={params['propertyTypeGroup']}&regionCode={params['regionCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_selangor_flat_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_selangor_flat_data.csv'


In [32]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "_freetextDisplay": "Selangor",
        "isCommercial": "false",
        "listingType": "sale",
        "page": "1",  # Static page parameter
        "propertyTypeCode": ["SRES"],
        "propertyTypeGroup": "N",
        "regionCode": "45nk1"  # Selangor region code
    }
    # Build the URL with the pattern
    return f"{base_url}/{page}?_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&listingType={params['listingType']}&page=1&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]]) + \
           f"&propertyTypeGroup={params['propertyTypeGroup']}&regionCode={params['regionCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_selangor_servresi_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_selangor_servresi_data.csv'


### Selangor - House

In [27]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "listingType": "sale",
        "page": str(page),
        "regionCode": "45nk1",
        "_freetextDisplay": "Selangor",
        "isCommercial": "false",
        "propertyTypeGroup": "T",
        "propertyTypeCode": ["TER1", "TER15", "TER2", "TER25", "TER3", "TER35", 
                           "TER4", "TER45", "TERRA"]
    }
    # Build the URL with the new pattern
    return f"{base_url}?listingType={params['listingType']}&page={params['page']}&regionCode={params['regionCode']}&_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&propertyTypeGroup={params['propertyTypeGroup']}&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]])

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_selangor_terrhouse_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_selangor_terrhouse_data.csv'


In [28]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "listingType": "sale",
        "page": str(page),
        "regionCode": "45nk1",
        "_freetextDisplay": "Selangor",
        "isCommercial": "false",
        "propertyTypeGroup": "T",
        "propertyTypeCode": ["TOWN"]
    }
    # Build the URL with the new pattern
    return f"{base_url}?listingType={params['listingType']}&page={params['page']}&regionCode={params['regionCode']}&_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&propertyTypeGroup={params['propertyTypeGroup']}&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]])

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_selangor_townhouse_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_selangor_townhouse_data.csv'


In [26]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "listingType": "sale",
        "page": str(page),
        "regionCode": "45nk1",
        "_freetextDisplay": "Selangor",
        "isCommercial": "false",
        "propertyTypeGroup": "S",  # S for Semi-Detached
        "propertyTypeCode": "SEMI"  # Only SEMI-D
    }
    return f"{base_url}?listingType={params['listingType']}&page={params['page']}&regionCode={params['regionCode']}&_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&propertyTypeGroup={params['propertyTypeGroup']}&propertyTypeCode={params['propertyTypeCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_selangor_semidhouse_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_selangor_semidhouse_data.csv'


In [37]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "listingType": "sale",
        "page": str(page),
        "regionCode": "45nk1",
        "_freetextDisplay": "Selangor",
        "isCommercial": "false",
        "propertyTypeGroup": "B",  # B for Bungalow
        "propertyTypeCode": "BUNG"  # Bungalow specific code
    }
    return f"{base_url}?listingType={params['listingType']}&page={params['page']}&regionCode={params['regionCode']}&_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&propertyTypeGroup={params['propertyTypeGroup']}&propertyTypeCode={params['propertyTypeCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_selangor_bungalow_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_selangor_bungalow_data.csv'


### KL - Apartment & Condominium

In [38]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "_freetextDisplay": "Kuala Lumpur",
        "isCommercial": "false",
        "listingType": "sale",
        "page": "1",
        "propertyTypeCode": ["APT"],
        "propertyTypeGroup": "N",
        "regionCode": "58jok"
    }
    return f"{base_url}/{page}?_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&listingType={params['listingType']}&page=1&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]]) + \
           f"&propertyTypeGroup={params['propertyTypeGroup']}&regionCode={params['regionCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_kl_apartment_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_kl_apartment_data.csv'


In [39]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "_freetextDisplay": "Kuala Lumpur",
        "isCommercial": "false",
        "listingType": "sale",
        "page": "1",  # Static page parameter
        "propertyTypeCode": ["APT"],
        "propertyTypeGroup": "N",
        "regionCode": "58jok"  # Region code
    }
    # Build the URL with the pattern
    return f"{base_url}/{page}?_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&listingType={params['listingType']}&page=1&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]]) + \
           f"&propertyTypeGroup={params['propertyTypeGroup']}&regionCode={params['regionCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_kl_condominium_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_kl_condominium_data.csv'


In [40]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "_freetextDisplay": "Kuala Lumpur",
        "isCommercial": "false",
        "listingType": "sale",
        "page": "1",  # Static page parameter
        "propertyTypeCode": ["FLAT"],
        "propertyTypeGroup": "N",
        "regionCode": "58jok"  # Region code
    }
    # Build the URL with the pattern
    return f"{base_url}/{page}?_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&listingType={params['listingType']}&page=1&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]]) + \
           f"&propertyTypeGroup={params['propertyTypeGroup']}&regionCode={params['regionCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_kl_flat_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_kl_flat_data.csv'


In [41]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "_freetextDisplay": "Kuala Lumpur",
        "isCommercial": "false",
        "listingType": "sale",
        "page": "1",  # Static page parameter
        "propertyTypeCode": ["SRES"],
        "propertyTypeGroup": "N",
        "regionCode": "58jok"  # region code
    }
    # Build the URL with the pattern
    return f"{base_url}/{page}?_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&listingType={params['listingType']}&page=1&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]]) + \
           f"&propertyTypeGroup={params['propertyTypeGroup']}&regionCode={params['regionCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_kl_servresi_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_kl_servresi_data.csv'


### KL - House

In [42]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "listingType": "sale",
        "page": str(page),
        "regionCode": "58jok",
        "_freetextDisplay": "Kuala Lumpur",
        "isCommercial": "false",
        "propertyTypeGroup": "T",
        "propertyTypeCode": ["TER1", "TER15", "TER2", "TER25", "TER3", "TER35", 
                           "TER4", "TER45", "TERRA"]
    }
    # Build the URL with the new pattern
    return f"{base_url}?listingType={params['listingType']}&page={params['page']}&regionCode={params['regionCode']}&_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&propertyTypeGroup={params['propertyTypeGroup']}&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]])

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_kl_terrhouse_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_kl_terrhouse_data.csv'


In [43]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "listingType": "sale",
        "page": str(page),
        "regionCode": "58jok",
        "_freetextDisplay": "Kuala Lumpur",
        "isCommercial": "false",
        "propertyTypeGroup": "T",
        "propertyTypeCode": ["TOWN"]
    }
    # Build the URL with the new pattern
    return f"{base_url}?listingType={params['listingType']}&page={params['page']}&regionCode={params['regionCode']}&_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&propertyTypeGroup={params['propertyTypeGroup']}&" + \
           "&".join([f"propertyTypeCode={code}" for code in params["propertyTypeCode"]])

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_kl_townhouse_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_kl_townhouse_data.csv'


In [44]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "listingType": "sale",
        "page": str(page),
        "regionCode": "58jok",
        "_freetextDisplay": "Kuala Lumpur",
        "isCommercial": "false",
        "propertyTypeGroup": "S",  # S for Semi-Detached
        "propertyTypeCode": "SEMI"  # Only SEMI-D
    }
    return f"{base_url}?listingType={params['listingType']}&page={params['page']}&regionCode={params['regionCode']}&_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&propertyTypeGroup={params['propertyTypeGroup']}&propertyTypeCode={params['propertyTypeCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_kl_semidhouse_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_kl_semidhouse_data.csv'


In [45]:
def get_url(page):
    base_url = "https://www.propertyguru.com.my/property-for-sale"
    params = {
        "listingType": "sale",
        "page": str(page),
        "regionCode": "58jok",
        "_freetextDisplay": "Kuala Lumpur",
        "isCommercial": "false",
        "propertyTypeGroup": "B",  # B for Bungalow
        "propertyTypeCode": "BUNG"  # Bungalow specific code
    }
    return f"{base_url}?listingType={params['listingType']}&page={params['page']}&regionCode={params['regionCode']}&_freetextDisplay={params['_freetextDisplay']}&isCommercial={params['isCommercial']}&propertyTypeGroup={params['propertyTypeGroup']}&propertyTypeCode={params['propertyTypeCode']}"

# Scrap
if __name__ == "__main__":
    scrape_batch(1, 9, 'propertyguru_kl_bungalow_data.csv')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
✅ Saved batch 1-9 to 'propertyguru_kl_bungalow_data.csv'


## Scrap Facilities

In [7]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

def init_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--log-level=3")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

def scrape_facilities(url, headless=True):
    driver = None
    try:
        driver = init_driver(headless)
        driver.get(url)
        
        # Wait for page to fully load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
        )
        
        # Scroll to facilities section
        driver.execute_script("window.scrollTo(0, 1500);")
        time.sleep(1)
        
        # Find section containing "Common facilities" text
        sections = driver.find_elements(By.CSS_SELECTOR, "section.property-facilities-section, section.property-amenities-section")
        target_section = None
        
        for section in sections:
            try:
                # Check if section contains "Common facilities" text
                header = section.find_element(By.XPATH, ".//*[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'common facilities')]")
                target_section = section
                break
            except NoSuchElementException:
                continue
        
        if target_section:
            facility_elements = target_section.find_elements(By.CSS_SELECTOR, ".property-amenities__row-item__value")
            facilities = [elem.text.strip() for elem in facility_elements if elem.text.strip()]
            return ", ".join(facilities) if facilities else "Not Listed"
        else:
            return "Not Listed"
            
    except Exception as e:
        print(f"[Error] {str(e)} in {url}")
        return "Not Listed"
    finally:
        if driver:
            driver.quit()

def add_facilities_to_dataset(input_csv, output_csv, headless=True):
    df = pd.read_csv(input_csv)
    
    # Create a copy of the DataFrame to save progress
    temp_df = df.copy()
    
    for idx, url in enumerate(df['URL']):
        print(f"Scraping ({idx + 1}/{len(df)}): {url}")
        facilities = scrape_facilities(url, headless)
        print(f"  Facilities: {facilities}")
        
        # Update temp DataFrame and save
        temp_df.at[idx, 'Facilities'] = facilities
        if (idx + 1) % 20 == 0:  # Save progress periodically
            temp_df.to_csv(output_csv, index=False)
            print(f"  Progress saved at {idx + 1} listings")
        
        # Random delay between requests
        delay = random.uniform(2.0, 4.0)  # Reduced delay
        print(f"  Waiting {delay:.1f} seconds...")
        time.sleep(delay)
    
    # Final save
    temp_df.to_csv(output_csv, index=False)
    print(f"Scraping completed. Output saved to {output_csv}")

In [50]:
if __name__ == "__main__":
    # scrap KL facilities
    input_csv = "propertyguru_data.csv"
    output_csv = "propertyguru_data_with_facilities.csv"
    add_facilities_to_dataset(input_csv, output_csv, headless=True)

Scraping (1/2724): https://www.propertyguru.com.my/property-listing/putra-permai-block-b-for-sale-by-kean-phang-60255130
  Facilities: Not Listed
  Waiting 3.3 seconds...
Scraping (2/2724): https://www.propertyguru.com.my/property-listing/project/rimbun-saujana-residency-for-sale-by-saujana-development-sdn-bhd-42341737#7268
  Facilities: 24 hours security, Bbq, Covered car park, Drop off point
  Waiting 2.5 seconds...
Scraping (3/2724): https://www.propertyguru.com.my/property-listing/taman-kajang-sentral-for-sale-by-quek-60270048
  Facilities: Not Listed
  Waiting 3.7 seconds...
Scraping (4/2724): https://www.propertyguru.com.my/property-listing/vista-saujana-for-sale-by-celine-yeo-43374096
  Facilities: 24 hours security, Badminton hall, Bbq, Covered car park
  Waiting 2.9 seconds...
Scraping (5/2724): https://www.propertyguru.com.my/property-listing/suria-court-for-sale-by-lucy-khoo-43023381
  Facilities: 24 hours security, Covered car park
  Waiting 3.1 seconds...
Scraping (6/2724)

Scraping (35/2724): https://www.propertyguru.com.my/property-listing/saville-d-lake-for-sale-by-alice-kuon-60365773
  Facilities: Bbq, Cafes, Gym, Jogging track
  Waiting 3.5 seconds...
Scraping (36/2724): https://www.propertyguru.com.my/property-listing/palm-ville-for-sale-by-denson-yong-31707091
  Facilities: 24 hours security, Badminton hall, Bbq, Covered car park
  Waiting 3.2 seconds...
Scraping (37/2724): https://www.propertyguru.com.my/property-listing/d7-d-seven-lagoon-perdana-for-sale-by-melinda-goh-60341980
  Facilities: 24 hours security, Badminton hall, Gym, Multi-purpose hall
  Waiting 4.0 seconds...
Scraping (38/2724): https://www.propertyguru.com.my/property-listing/desa-mas-apartment-rawang-for-sale-by-andy-yap-34590841
  Facilities: Not Listed
  Waiting 2.7 seconds...
Scraping (39/2724): https://www.propertyguru.com.my/property-listing/sri-bahagia-court-for-sale-by-najibah-sahari-500026304
  Facilities: 24 hours security, Bbq, Covered car park, Lounge
  Waiting 3.6 sec

Scraping (75/2724): https://www.propertyguru.com.my/property-listing/orchis-apartment-for-sale-by-kho-daniel-60365277
  Facilities: 24 hours security, Perimeter fencing, Playground
  Waiting 3.8 seconds...
Scraping (76/2724): https://www.propertyguru.com.my/property-listing/saville-d-lake-for-sale-by-elvis-ho-60007640
  Facilities: Bbq, Cafes, Gym, Jogging track
  Waiting 2.4 seconds...
Scraping (77/2724): https://www.propertyguru.com.my/property-listing/taman-aman-perdana-for-sale-by-shahidan-ismail-60365216
  Facilities: Surau (male & female)
  Waiting 3.7 seconds...
Scraping (78/2724): https://www.propertyguru.com.my/property-listing/masreca-n19eteen-for-sale-by-zf-chua-42606709
  Facilities: 24 hours security, Basketball court, Parking, Tennis courts
  Waiting 4.0 seconds...
Scraping (79/2724): https://www.propertyguru.com.my/property-listing/sri-ehsan-for-sale-by-tiffany-meng-60348561
  Facilities: 24 hours security, Playground
  Waiting 3.4 seconds...
Scraping (80/2724): https://

  Facilities: Parking
  Waiting 3.7 seconds...
Scraping (117/2724): https://www.propertyguru.com.my/property-listing/vistaria-for-sale-by-bryan-ching-43324680
  Facilities: 24 hours security, Covered car park, Playground
  Waiting 2.9 seconds...
Scraping (118/2724): https://www.propertyguru.com.my/property-listing/vistaria-for-sale-by-bryan-ching-42443978
  Facilities: 24 hours security, Covered car park, Playground
  Waiting 3.5 seconds...
Scraping (119/2724): https://www.propertyguru.com.my/property-listing/pangsapuri-carmila-for-sale-by-michelle-chow-60004734
  Facilities: Bbq, Lounge, Nursery, Parking
  Waiting 3.9 seconds...
Scraping (120/2724): https://www.propertyguru.com.my/property-listing/d-shire-villa-for-sale-by-michelle-chow-60004687
  Facilities: 24 hours security, Cafes, Clubhouse, Covered car park
  Progress saved at 120 listings
  Waiting 2.4 seconds...
Scraping (121/2724): https://www.propertyguru.com.my/property-listing/rebana-flat-for-sale-by-jamie-loke-60364680
  F

Scraping (158/2724): https://www.propertyguru.com.my/property-listing/seri-baiduri-for-sale-by-cc-hong-43456684
  Facilities: 24 hours security, Swimming pool
  Waiting 2.6 seconds...
Scraping (159/2724): https://www.propertyguru.com.my/property-listing/mutiara-hilltop-for-sale-by-ho-km-500025009
  Facilities: Basketball court, Bbq, Drop off point, Gym
  Waiting 2.4 seconds...
Scraping (160/2724): https://www.propertyguru.com.my/property-listing/sri-kenanga-for-sale-by-peter-kok-60324593
  Facilities: 24 hours security, Covered car park, Lounge, Nursery
  Progress saved at 160 listings
  Waiting 2.6 seconds...
Scraping (161/2724): https://www.propertyguru.com.my/property-listing/palma-puteri-for-sale-by-serene-ong-60110920
  Facilities: 24 hours security, Covered car park, Playground
  Waiting 2.5 seconds...
Scraping (162/2724): https://www.propertyguru.com.my/property-listing/de-bayu-apartment-for-sale-by-arif-mustapha-43027310
  Facilities: 24 hours security, Gym, Jogging track, Play

...

  Facilities: Parking
  Waiting 3.8 seconds...
Scraping (890/2724): https://www.propertyguru.com.my/property-listing/eco-forest-ebonylane-artisan-home-for-sale-by-ck-law-42857833
  Facilities: 24 hours security
  Waiting 3.4 seconds...
Scraping (891/2724): https://www.propertyguru.com.my/property-listing/terrace-house-at-puchong-for-sale-by-ronald-soo-37958257
  Facilities: 24 hours security, Basketball court, Parking
  Waiting 3.0 seconds...
Scraping (892/2724): https://www.propertyguru.com.my/property-listing/goodview-heights-for-sale-by-aivan-ng-60307978
  Facilities: Not Listed
  Waiting 2.9 seconds...
Scraping (893/2724): https://www.propertyguru.com.my/property-listing/taman-minang-cheras-for-sale-by-eddie-ng-fui-siang-43421927
  Facilities: Not Listed
  Waiting 3.1 seconds...
Scraping (894/2724): https://www.propertyguru.com.my/property-listing/m-legasi-for-sale-by-kevin-lee-60051521
  Facilities: 24 hours security
  Waiting 2.8 seconds...
Scraping (895/2724): https://www.proper

  Facilities: 24 hours security, Bbq, Community garden, Jogging track
  Progress saved at 920 listings
  Waiting 3.2 seconds...
Scraping (921/2724): https://www.propertyguru.com.my/property-listing/project/pkns-ixora-for-sale-by-perbadanan-kemajuan-negeri-selangor-pkns-43089628?pixelUrl=https%3A%2F%2Fe-11135.adzerk.net%2Fe.gif%3Fe%3DeyJ2IjoiMS4xMyIsImF2Ijo2MDQ5ODQ1LCJhdCI6MTYzLCJidCI6MCwiY20iOjYzMjI4ODY5MSwiY2giOjU5NTM0LCJjayI6e30sImNyIjo3MTE2NTMxODAsImRpIjoiZmNhYjc1NjVjNGM2NDUxZGFkN2I4YWU1OWQ5NmI2NmQiLCJkaiI6MiwiaWkiOiIwNWFlM2I4OGEyZGQ0NjJiOGJhZDE2YmQxOWNiZmRiYiIsImRtIjozLCJmYyI6ODg1OTM2Njc3LCJmbCI6ODM1MTAyOTU3LCJpcCI6IjYwLjQ4LjE2OC45MyIsImt3IjoiNDVuazEsbGlzdGluZy1yZXNpZGVudGlhbCxsaXN0aW5nLXNhbGUtcHJvcC10IiwibWsiOiJsaXN0aW5nLXNhbGUtcHJvcC10LDQ1bmsxIiwibnciOjExMTM1LCJwYyI6MjUwMCwib3AiOjI1MDAsIm1wIjoyNTAwLCJlYyI6MjUwMCwiZ20iOjAsImVwIjpudWxsLCJwciI6MjM1NzE0LCJydCI6MiwicnMiOjUwMCwic2EiOiI4MSIsInNiIjoiaS0wOGZkNGE1MzBlYzA2NzliMCIsInNwIjozNTc2MTcsInN0IjoxMjM0NTU1LCJ1ayI6IjI5OTJjNDMwLTExZDItN

  Facilities: Badminton hall, Basketball court, Playground
  Waiting 2.4 seconds...
Scraping (947/2724): https://www.propertyguru.com.my/property-listing/antilla-lagoon-villas-for-sale-by-lily-yap-dh-60174833
  Facilities: Parking, Playground
  Waiting 3.7 seconds...
Scraping (948/2724): https://www.propertyguru.com.my/property-listing/bandar-damai-perdana-for-sale-by-li-yen-wong-60360170
  Facilities: Not Listed
  Waiting 3.6 seconds...
Scraping (949/2724): https://www.propertyguru.com.my/property-listing/project/aludra-residensi-for-sale-by-perbadanan-kemajuan-negeri-selangor-pkns-43399246?pixelUrl=https%3A%2F%2Fe-11135.adzerk.net%2Fe.gif%3Fe%3DeyJ2IjoiMS4xMyIsImF2Ijo2MDQ5Nzk0LCJhdCI6MTYzLCJidCI6MCwiY20iOjY1MTA5MDkwMiwiY2giOjU5NTM0LCJjayI6e30sImNyIjo3NDE4MDE2OTYsImRpIjoiMTkzOTc3NDY2NjFjNDRmMWIwYjgzNDE2YjA1Y2IxODEiLCJkaiI6MiwiaWkiOiJiZjg1ZDdhNzNiZjQ0NGU4ODEwMDU5OTRmMjA0M2VjYSIsImRtIjozLCJmYyI6OTE2MDkzMDU0LCJmbCI6ODUzOTU0NzE4LCJpcCI6IjYwLjQ4LjE2OC45MyIsImt3IjoiNDVuazEsbGlzdGluZy1yZXNpZ

Scraping (982/2724): https://www.propertyguru.com.my/property-listing/rawang-for-sale-by-elvin-soh-43116045
  Facilities: Covered car park, Jogging track, Pet-friendly facilities
  Waiting 3.5 seconds...
Scraping (983/2724): https://www.propertyguru.com.my/property-listing/townhouse-at-country-heights-for-sale-by-nick-chan-42618692
  Facilities: Parking
  Waiting 2.3 seconds...
Scraping (984/2724): https://www.propertyguru.com.my/property-listing/ariza-garden-villa-for-sale-by-ethan-swe-60360386
  Facilities: 24 hours security, Parking
  Waiting 2.6 seconds...
Scraping (985/2724): https://www.propertyguru.com.my/property-listing/amansiara-for-sale-by-danny-wong-43213261
  Facilities: 24 hours security, Bbq, Clubhouse, Gym
  Waiting 3.7 seconds...
Scraping (986/2724): https://www.propertyguru.com.my/property-listing/prima-court-for-sale-by-muhd-fahmie-60042801
  Facilities: 24 hours security, Bbq, Covered car park, Lounge
  Waiting 3.7 seconds...
Scraping (987/2724): https://www.propert

KeyboardInterrupt: 

**Code to conitnue scraping**

In [11]:
def add_facilities_to_dataset(input_csv, output_csv, headless=True, start_idx=0):
    df = pd.read_csv(input_csv)

    for idx in range(start_idx, len(df)):
        url = df.loc[idx, 'URL']
        if pd.notna(df.loc[idx, 'Facilities']):  
            # Skip already-scraped listings
            continue

        print(f"Scraping ({idx + 1}/{len(df)}): {url}")
        facilities = scrape_facilities(url, headless)
        print(f"  Facilities: {facilities}")

        df.at[idx, 'Facilities'] = facilities

        if (idx + 1) % 20 == 0:
            df.to_csv(output_csv, index=False)
            print(f"  Progress saved at {idx + 1} listings")

        delay = random.uniform(2.0, 4.0)
        print(f"  Waiting {delay:.1f} seconds...")
        time.sleep(delay)

    df.to_csv(output_csv, index=False)
    print(f"Scraping completed. Output saved to {output_csv}")


In [12]:
add_facilities_to_dataset("propertyguru_data_with_facilities.csv", "propertyguru_data_with_facilities_v2.csv", headless=True, start_idx=1020)


Scraping (1021/2724): https://www.propertyguru.com.my/property-listing/taragon-puteri-cheras-for-sale-by-chen-kar-mun-500019699
  Facilities: 24 hours security, Bbq, Jogging track, Perimeter fencing
  Waiting 2.0 seconds...
Scraping (1022/2724): https://www.propertyguru.com.my/property-listing/kota-kemuning-for-sale-by-wilson-ng-60355504
  Facilities: Parking
  Waiting 3.4 seconds...
Scraping (1023/2724): https://www.propertyguru.com.my/property-listing/kota-kemuning-for-sale-by-wilson-ng-60355611
  Facilities: Parking
  Waiting 3.3 seconds...
Scraping (1024/2724): https://www.propertyguru.com.my/property-listing/jenderam-hilir-for-sale-by-fb-chong-60048423
  Facilities: Not Listed
  Waiting 2.5 seconds...
Scraping (1025/2724): https://www.propertyguru.com.my/property-listing/bayan-villa-seri-kembangan-for-sale-by-nicholas-lew-42362094
  Facilities: Parking
  Waiting 2.7 seconds...
Scraping (1026/2724): https://www.propertyguru.com.my/property-listing/bayan-villa-seri-kembangan-for-sal

  Facilities: 24 hours security, Covered car park, Playground
  Waiting 3.5 seconds...
Scraping (1063/2724): https://www.propertyguru.com.my/property-listing/parkville-for-sale-by-wind-cheang-40704426
  Facilities: 24 hours security, Parking
  Waiting 3.0 seconds...
Scraping (1064/2724): https://www.propertyguru.com.my/property-listing/mutiara-tropicana-for-sale-by-zuraidah-dan-41796551
  Facilities: 24 hours security, Playground, Surau (male & female)
  Waiting 2.3 seconds...
Scraping (1065/2724): https://www.propertyguru.com.my/property-listing/parkville-for-sale-by-wind-cheang-42899187
  Facilities: 24 hours security, Parking
  Waiting 3.2 seconds...
Scraping (1066/2724): https://www.propertyguru.com.my/property-listing/parkville-for-sale-by-wind-cheang-39717076
  Facilities: 24 hours security, Parking
  Waiting 3.4 seconds...
Scraping (1067/2724): https://www.propertyguru.com.my/property-listing/parkville-for-sale-by-wind-cheang-41839041
  Facilities: 24 hours security, Parking
  W

[Error] Message: timeout: Timed out receiving message from renderer: 29.337
  (Session info: chrome=139.0.7258.128)
Stacktrace:
	GetHandleVerifier [0x0x7ff64a326b25+79621]
	GetHandleVerifier [0x0x7ff64a326b80+79712]
	(No symbol) [0x0x7ff64a0bc0ea]
	(No symbol) [0x0x7ff64a0a9a0c]
	(No symbol) [0x0x7ff64a0a96fa]
	(No symbol) [0x0x7ff64a0a72c8]
	(No symbol) [0x0x7ff64a0a7d2f]
	(No symbol) [0x0x7ff64a0b694e]
	(No symbol) [0x0x7ff64a0cc911]
	(No symbol) [0x0x7ff64a0d3a6a]
	(No symbol) [0x0x7ff64a0a84cd]
	(No symbol) [0x0x7ff64a0cc645]
	(No symbol) [0x0x7ff64a16373d]
	(No symbol) [0x0x7ff64a13af13]
	(No symbol) [0x0x7ff64a104151]
	(No symbol) [0x0x7ff64a104ee3]
	GetHandleVerifier [0x0x7ff64a5e683d+2962461]
	GetHandleVerifier [0x0x7ff64a5e0b5d+2938685]
	GetHandleVerifier [0x0x7ff64a5ff71d+3064573]
	GetHandleVerifier [0x0x7ff64a340c6e+186446]
	GetHandleVerifier [0x0x7ff64a348a3f+218655]
	GetHandleVerifier [0x0x7ff64a32f914+115956]
	GetHandleVerifier [0x0x7ff64a32fac9+116393]
	GetHandleVerifier

  Facilities: 24 hours security, Covered car park, Playground
  Waiting 2.1 seconds...
Scraping (1138/2724): https://www.propertyguru.com.my/property-listing/bukit-sungai-long-1-for-sale-by-shanny-tan-42522453
  Facilities: 24 hours security, Covered car park, Jogging track, Playground
  Waiting 3.1 seconds...
Scraping (1139/2724): https://www.propertyguru.com.my/property-listing/taming-indah-1-bandar-sungai-long-for-sale-by-shanny-tan-41638344
  Facilities: 24 hours security, Covered car park, Playground
  Waiting 3.7 seconds...
Scraping (1140/2724): https://www.propertyguru.com.my/property-listing/taming-mutiara-for-sale-by-shanny-tan-43260226
  Facilities: 24 hours security
  Progress saved at 1140 listings
  Waiting 3.0 seconds...
Scraping (1141/2724): https://www.propertyguru.com.my/property-listing/semanja-kajang-for-sale-by-sean-33479465
  Facilities: 24 hours security, Jogging track, Playground
  Waiting 3.2 seconds...
Scraping (1142/2724): https://www.propertyguru.com.my/prope

  Facilities: Basketball court, Bbq, Bus stop, Cafes
  Waiting 3.1 seconds...
Scraping (1172/2724): https://www.propertyguru.com.my/property-listing/setia-alam-for-sale-by-marcus-tan-60191676
  Facilities: Landscaped garden, Multi-purpose hall, Playground, Retail stores
  Waiting 4.0 seconds...
Scraping (1173/2724): https://www.propertyguru.com.my/property-listing/temasya-anggun-temasya-glenmarie-for-sale-by-darren-ng-60177828
  Facilities: 24 hours security, Jogging track, Playground
  Waiting 2.1 seconds...
Scraping (1174/2724): https://www.propertyguru.com.my/property-listing/greenhill-residence-for-sale-by-fazie-noor-43297548
  Facilities: Parking
  Waiting 3.5 seconds...
Scraping (1175/2724): https://www.propertyguru.com.my/property-listing/double-storey-semi-d-anjung-suasana-bandar-seri-putra-bangi-for-sale-by-fazie-noor-60027301
  Facilities: Not Listed
  Waiting 2.5 seconds...
Scraping (1176/2724): https://www.propertyguru.com.my/property-listing/kingsley-hills-putra-heights-fo

...

Scraping (2532/2724): https://www.propertyguru.com.my/property-listing/sunway-montana-for-sale-by-william-lua-42729447
  Facilities: 24 hours security, Clubhouse, Playground
  Waiting 3.6 seconds...
Scraping (2533/2724): https://www.propertyguru.com.my/property-listing/taman-yarl-for-sale-by-priscilla-42616466
  Facilities: Not Listed
  Waiting 3.1 seconds...
Scraping (2534/2724): https://www.propertyguru.com.my/property-listing/taman-melawati-for-sale-by-salmi-binti-aripin-39408280
  Facilities: 24 hours security, Playground
  Waiting 2.5 seconds...
Scraping (2535/2724): https://www.propertyguru.com.my/property-listing/desa-tiara-kuchai-lama-for-sale-by-yenn-chong-34988755
  Facilities: 24 hours security
  Waiting 3.9 seconds...
Scraping (2536/2724): https://www.propertyguru.com.my/property-listing/levenue-for-sale-by-leon-liew-500015057
  Facilities: 24 hours security, Clubhouse, Parking, Playground
  Waiting 3.9 seconds...
Scraping (2537/2724): https://www.propertyguru.com.my/proper

Scraping (2571/2724): https://www.propertyguru.com.my/property-listing/kl-city-rental-cover-instalment-roi10-walk-to-mrt-lrt-ktm-provide-shuttle-bus-for-sale-by-wei-nie-43061893
  Facilities: 24 hours security, Badminton hall, Basketball court, Bbq
  Waiting 3.0 seconds...
Scraping (2572/2724): https://www.propertyguru.com.my/property-listing/bungalow-at-damansara-heights-for-sale-by-jason-lee-36155350
  Facilities: Not Listed
  Waiting 2.5 seconds...
Scraping (2573/2724): https://www.propertyguru.com.my/property-listing/taman-tun-dr-ismail-for-sale-by-yin-peng-37177004
  Facilities: Not Listed
  Waiting 3.1 seconds...
Scraping (2574/2724): https://www.propertyguru.com.my/property-listing/taman-tun-dr-ismail-for-sale-by-yin-peng-42954366
  Facilities: Not Listed
  Waiting 2.6 seconds...
Scraping (2575/2724): https://www.propertyguru.com.my/property-listing/taman-tun-dr-ismail-for-sale-by-yin-peng-41719042
  Facilities: Not Listed
  Waiting 3.4 seconds...
Scraping (2576/2724): https://w

Scraping (2613/2724): https://www.propertyguru.com.my/property-listing/bungalow-at-jalan-bruas-damansara-heights-for-sale-by-jacky-lee-wei-jie-500015962
  Facilities: Not Listed
  Waiting 3.3 seconds...
Scraping (2614/2724): https://www.propertyguru.com.my/property-listing/jalan-awan-kecil-taman-oug-for-sale-by-simon-ng-41882038
  Facilities: Not Listed
  Waiting 3.5 seconds...
Scraping (2615/2724): https://www.propertyguru.com.my/property-listing/tijani-2-south-for-sale-by-teoh-yie-hao-41213642
  Facilities: 24 hours security, Clubhouse, Parking, Perimeter fencing
  Waiting 3.0 seconds...
Scraping (2616/2724): https://www.propertyguru.com.my/property-listing/country-heights-damansara-for-sale-by-william-43218868
  Facilities: 24 hours security, Clubhouse, Community garden, Parking
  Waiting 2.3 seconds...
Scraping (2617/2724): https://www.propertyguru.com.my/property-listing/country-heights-damansara-for-sale-by-william-43219012
  Facilities: 24 hours security, Clubhouse, Community ga

Scraping (2654/2724): https://www.propertyguru.com.my/property-listing/bangsar-for-sale-by-nick-yeap-25242740
  Facilities: Not Listed
  Waiting 2.3 seconds...
Scraping (2655/2724): https://www.propertyguru.com.my/property-listing/kiara-hills-for-sale-by-nick-yeap-37769018
  Facilities: 24 hours security, Cafes, Parking, Swimming pool
  Waiting 2.9 seconds...
Scraping (2656/2724): https://www.propertyguru.com.my/property-listing/hamton-kemensah-for-sale-by-zheng-hong-tan-34450814
  Facilities: 24 hours security, Covered car park, Jacuzzi, Playground
  Waiting 3.7 seconds...
Scraping (2657/2724): https://www.propertyguru.com.my/property-listing/damansara-heights-for-sale-by-james-thong-42259514
  Facilities: Not Listed
  Waiting 4.0 seconds...
Scraping (2658/2724): https://www.propertyguru.com.my/property-listing/taman-titiwangsa-for-sale-by-maggie-kho-500025696
  Facilities: Not Listed
  Waiting 2.2 seconds...
Scraping (2659/2724): https://www.propertyguru.com.my/property-listing/sri-p

  Facilities: 24 hours security, Bbq, Clubhouse, Covered car park
  Waiting 3.1 seconds...
Scraping (2696/2724): https://www.propertyguru.com.my/property-listing/flora-murni-for-sale-by-elaine-chong-elle-60052535
  Facilities: 24 hours security, Bbq, Gym, Jacuzzi
  Waiting 2.3 seconds...
Scraping (2697/2724): https://www.propertyguru.com.my/property-listing/dh-12000sf-land-klcc-view-6-rooms-park-6-cars-for-sale-by-elaine-chong-elle-60050478
  Facilities: Swimming pool
  Waiting 2.9 seconds...
Scraping (2698/2724): https://www.propertyguru.com.my/property-listing/dh-elevated-home-nice-breeze-for-sale-by-elaine-chong-elle-60052506
  Facilities: Not Listed
  Waiting 3.0 seconds...
Scraping (2699/2724): https://www.propertyguru.com.my/property-listing/laman-setiakasih-8-for-sale-by-elaine-chong-elle-60052490
  Facilities: 24 hours security, Parking, Perimeter fencing, Swimming pool
  Waiting 3.4 seconds...
Scraping (2700/2724): https://www.propertyguru.com.my/property-listing/country-heigh

## Data Overview

In [49]:
# Selangor datasets
df_pguru1 = pd.read_csv('propertyguru_selangor_apartment_data.csv')
df_pguru2 = pd.read_csv('propertyguru_selangor_condominium_data.csv')
df_pguru3 = pd.read_csv('propertyguru_selangor_flat_data.csv')
df_pguru4 = pd.read_csv('propertyguru_selangor_servresi_data.csv')
df_pguru5 = pd.read_csv('propertyguru_selangor_terrhouse_data.csv')
df_pguru6 = pd.read_csv('propertyguru_selangor_townhouse_data.csv')
df_pguru7 = pd.read_csv('propertyguru_selangor_semidhouse_data.csv')

# KL datasets (same naming pattern)
df_pguru8  = pd.read_csv('propertyguru_kl_apartment_data.csv')
df_pguru9  = pd.read_csv('propertyguru_kl_condominium_data.csv')
df_pguru10 = pd.read_csv('propertyguru_kl_flat_data.csv')
df_pguru11 = pd.read_csv('propertyguru_kl_servresi_data.csv')
df_pguru12 = pd.read_csv('propertyguru_kl_terrhouse_data.csv')
df_pguru13 = pd.read_csv('propertyguru_kl_townhouse_data.csv')
df_pguru14 = pd.read_csv('propertyguru_kl_semidhouse_data.csv')
df_pguru15 = pd.read_csv('propertyguru_kl_bungalow_data.csv')

# Combine all DataFrames into a list
dfs = [
    df_pguru1, df_pguru2, df_pguru3, df_pguru4, df_pguru5, df_pguru6, df_pguru7,
    df_pguru8, df_pguru9, df_pguru10, df_pguru11, df_pguru12, df_pguru13, df_pguru14, df_pguru15
]

# Show shape for each
for i, df in enumerate(dfs, start=1):
    print(f"df_pguru{i} shape: {df.shape}")

# Total rows
total_rows = sum(df.shape[0] for df in dfs)
print("\nTotal rows across all datasets:", total_rows)

df_pguru1 shape: (180, 9)
df_pguru2 shape: (179, 9)
df_pguru3 shape: (180, 9)
df_pguru4 shape: (179, 9)
df_pguru5 shape: (190, 9)
df_pguru6 shape: (186, 9)
df_pguru7 shape: (180, 9)
df_pguru8 shape: (184, 9)
df_pguru9 shape: (183, 9)
df_pguru10 shape: (184, 9)
df_pguru11 shape: (183, 9)
df_pguru12 shape: (180, 9)
df_pguru13 shape: (180, 9)
df_pguru14 shape: (180, 9)
df_pguru15 shape: (176, 9)

Total rows across all datasets: 2724


In [51]:
# Combine the datasets
df_all = pd.concat(dfs, ignore_index=True)

# Save as CSV
df_all.to_csv("propertyguru_data.csv", index=False)